# Assortative Matching With Coarse Types: Chi-Square Estimator of Deep Parameters given Stochastic Classification of Economics Departments by Placement Rates

James Yu, 30 November 2021 - modified by mike December

## VERSION: $n$, $m$ fixed, ratios endogenous

In [1]:
INSTALL_PACKAGES = false   # change this to true if you are running this notebook for the first time
YEAR_INTERVAL = 2003:2021  # change this to select the years of data to include in the estimation
NUMBER_OF_TYPES = 4        # change this to select the number of types to classify academic departments into
NUMBER_OF_SINKS = 1        # change this to 4 to use individual sink types
SAVE_TO_DATADASE = false   # change to true to save the type allocation to the database
TOTAL_DISTRIBUTIONS = NUMBER_OF_TYPES + NUMBER_OF_SINKS;

In [2]:
import Pkg
for package in ["BlackBoxOptim", "Distributions", "ForwardDiff", "JSON", "Optim", "Quadrature", 
        "StatsPlots","DotEnv","MySQL","DBInterface","Tables"]
    if INSTALL_PACKAGES
        Pkg.add(package)
    end
end
using  JSON, HTTP, Distributions, DotEnv, MySQL, DBInterface, Tables ,Random

In [3]:
function get_data(url)
    resp = HTTP.get(url);
    es = String(resp.body)
    return placements = JSON.parse(es)
end

get_data (generic function with 1 method)

In [4]:
function find_inst(name::String,outcomes,to_from::Bool = false)
    check = false
    for outcome in outcomes
        if to_from == true
            if outcome["to_name"] == name
                println(outcome)
                check = true
            end
        else
            if outcome["from_institution_name"] == name
                println(outcome)
                check = true
            end
        end
    end
    if check == false
        println("Not found")
    end
end

find_inst (generic function with 2 methods)

This is the type allocation notebook, modified by Mike.

This part gets all the placement data using `https://support.econjobmarket.org/api/placement_data`

In [5]:
url = "https://support.econjobmarket.org/api/placement_data"
placements = get_data(url);
println(length(placements))
println(typeof(placements))

14301
Vector{Any}


In [6]:
placements = unique(placements);
println(length(placements))

14301


future debugging apparently there are 16 duplicates

In [7]:
const DB = DBInterface
cfg = DotEnv.config();

Find all the placements to assistant professor positions and put them in a set called academic_builder.
Put the other placements in a set called sink_builder

In [8]:
i = 0
oid_mapping = Dict{}()
institution_mapping = Dict{}()
academic = Set{}()
academic_to = Set{}()
academic_builder = Set{}()
sink_builder = Set{}()
for placement in placements
    if in(parse(Int64, placement["year"]), YEAR_INTERVAL)
        push!(academic, placement["from_institution_name"])
        push!(academic_to, placement["to_name"])
        oid_mapping[placement["from_oid"]] = placement["from_institution_id"]
        oid_mapping[placement["to_oid"]] = placement["to_institution_id"]
        institution_mapping[placement["from_institution_id"]] = placement["from_institution_name"]
        institution_mapping[placement["to_institution_id"]] = placement["to_name"]
        if placement["position_name"] == "Assistant Professor"
            push!(academic_builder, placement)
        else
            push!(sink_builder, placement)
        end
    end
end

println(length(academic_builder), " total assistant professor outcomes")
println(length(sink_builder), " other outcomes")
println(length(academic_builder)+length(sink_builder))


6945 total assistant professor outcomes
7041 other outcomes
13986


This is smaller than the total placements because it doesn't include placements outside the year interval.

This piece of code deals with teaching universities by checking if they ever graduated Ph.D. students:

In [9]:
tch_sink = Set{}() # sink of teaching universities that do not graduate PhDs
for key in academic_to
    if !(key in academic)
        push!(tch_sink, key)
    end
end

println(length(academic))
println(length(academic_to))
println(length(tch_sink))

691
1757
1168


these are institutions, not placements.  Logic relies on the fact that julia dicts and sets can't contain duplicates

The next piece of code sorts all the sink departments (except teaching universities, which are dealt with above) by category:

In [10]:
acd_sink = Set{}()
gov_sink = Set{}()
pri_sink = Set{}()

for outcome in sink_builder
    # CODE global academic, other_placements, pri_sink, gov_sink, acd_sink
    if outcome["recruiter_type"] in ["6","7"]
        # private sector: for and not for profit
        push!(pri_sink, string(outcome["to_name"], " (private sector)"))
    elseif outcome["recruiter_type"] == "5"
        # government institution
        push!(gov_sink, string(outcome["to_name"], " (public sector)"))
    else
        # everything else including terminal academic positions
        push!(acd_sink, string(outcome["to_name"], " (academic sink)"))
    end
end

println(length(acd_sink))
println(length(gov_sink))
println(length(pri_sink))

860
144
187


Now that we have five sets for every category of department, we can construct a matrix representing the placements between these departments:

In [11]:
institutions = vcat(collect(academic), collect(acd_sink), collect(gov_sink), collect(pri_sink), collect(tch_sink));


In [12]:

out = zeros(Int64, length(institutions), length(collect(academic)))
i = 0
for outcome in academic_builder
    i += 1
    out[findfirst(isequal(outcome["to_name"]), institutions), findfirst(isequal(outcome["from_institution_name"]), institutions)] += 1
end
for outcome in sink_builder
    i += 1
    keycheck = ""
    if outcome["recruiter_type"] in ["6", "7"]
        keycheck = string(outcome["to_name"], " (private sector)")
    elseif outcome["recruiter_type"] == "5"
        keycheck = string(outcome["to_name"], " (public sector)")
    else
        keycheck = string(outcome["to_name"], " (academic sink)")
    end
    #println(keycheck)
    #println(findfirst(isequal(keycheck), institutions))
    #println(outcome["from_institution_name"]," ",findfirst(isequal(outcome["from_institution_name"]), institutions))
    out[findfirst(isequal(keycheck), institutions), findfirst(isequal(outcome["from_institution_name"]), institutions)] += 1
end
println("Total number of outcomes: ", i)
println(length(gov_sink))
println(length(pri_sink))


Total number of outcomes: 13986
144
187


In [13]:
sum(out)

13986

Finally, we get to the estimator. For this estimate, we assume that each observed set of placement outcomes between any two pairs of departments is drawn from a distribution common to the "type" of the hiring department and the "type" of the graduating department. Here this distribution is assumed to be Poisson, in line with classical stochastic block models used for similar estimations in Karrer and Newman (2011) and Peixoto (2014).

Given a particular assignment of departments to types, and given the placement outcomes, a single round of estimation computes the mean number of applicants from any single type $t$ department that would be hired at a single type $t^\prime$ department and measures the probability that each independent observation was drawn from its corresponding mean. When summed together, the logarithms of the probabilities form a log-likelihood which can be used for maximum likelihood estimation.

In [14]:
function bucket_estimate(assign::Array{Int64}, A::Matrix{Int64}, num, numsink)
    b = zeros(Int64, size(A)[1], size(A)[2]) #this is the line that is moved
    T = zeros(num + numsink, num)
    count = zeros(num + numsink, num)
    for i in 1:size(A)[1], j in 1:size(A)[2]
         @inbounds val = (num + 1) * (assign[j] - 1) + assign[i]
         @inbounds b[i, j] = val
         @inbounds T[val] = ((T[val] * count[val]) + A[i, j]) / (count[val] + 1)
         @inbounds count[val] += 1
    end
    L = 0.0
    @simd for i in eachindex(A)
        @inbounds L += logpdf(Poisson(T[b[i]]), A[i])
    end
    return -L, T
end

bucket_estimate (generic function with 1 method)

In [15]:
function bucket_estimate_mem(assign::Array{Int64}, A::Matrix{Int64}, num, numsink,b)
    T = zeros(num + numsink, num)
    count = zeros(num + numsink, num)
    for i in 1:size(A)[1], j in 1:size(A)[2]
         @inbounds val = (num + 1) * (assign[j] - 1) + assign[i]
         @inbounds b[i, j] = val
         @inbounds T[val] = ((T[val] * count[val]) + A[i, j]) / (count[val] + 1)
         @inbounds count[val] += 1
    end
    L = 0.0
    @simd for i in eachindex(A)
        @inbounds L += logpdf(Poisson(T[b[i]]), A[i])
    end
    return -L, T
end

bucket_estimate_mem (generic function with 1 method)

Finally, we compute the maximum-likelihood estimated Poisson means by stochastically re-allocating departments to types and saving likelihood-improving re-allocations until no further re-allocations are found.

### To skip re-allocating, uncomment the following line instead if you have an allocation already. Do not run the cell after.

In [16]:
Random.seed!(1234)

TaskLocalRNG()

In [19]:
#est_alloc = JSON.parsefile("type_allocation.json");
function doit_mem(sample, academic_institutions, asink, gsink, psink, tsink, all_institutions, num, numsink)
    b = zeros(Int64, size(sample)[1], size(sample)[2])
    # some initial states
    current_allocation = Array{Int64}(undef, length(all_institutions))
    cur_objective = Inf
    best_mat = nothing
    cursor = 1
    for inst in academic_institutions
        current_allocation[cursor] = 1
        cursor += 1
    end
    # the sinks must stay in fixed types
    # this was built to support more sinks, but by default we only use one
    # change the "current_allocation[cursor] = ..." lines to group sinks together
    for key in asink # other academic
        current_allocation[cursor] = num + min(1, numsink)
        cursor += 1
    end
    for key in gsink # public sector
        current_allocation[cursor] = num + min(2, numsink)
        cursor += 1
    end
    for key in psink # private sector
        current_allocation[cursor] = num + min(3, numsink)
        cursor += 1
    end
    for key in tsink # assistant professor at teaching universities
        current_allocation[cursor] = num + min(4, numsink)
        cursor += 1
    end
    blankcount = 0

    # BEGIN MONTE CARLO REALLOCATION ROUTINE
    while true
        # attempt to reallocate academic institutions to a random spot
        temp_allocation = copy(current_allocation)
        k = rand(1:length(academic_institutions))
        @inbounds temp_allocation[k] = rand(delete!(Set(1:num), temp_allocation[k]))
        # check if the new assignment is better
        test_objective, estimated_means = bucket_estimate_mem(temp_allocation, sample, num, numsink,b)
        if test_objective < cur_objective
            print(test_objective, " ")
            blankcount = 0
            cur_objective = test_objective
            best_mat = estimated_means
            current_allocation = temp_allocation
        else
            blankcount += 1
            if blankcount % 1000 == 0
                print(blankcount, " ")
            end
        end
        if blankcount == 100000
            return cur_objective, best_mat, current_allocation
        end
    end
end


doit_mem (generic function with 1 method)

In [20]:
@time est_obj, est_mat, est_alloc = doit_mem(out, collect(academic), collect(acd_sink), collect(gov_sink), collect(pri_sink),
    collect(tch_sink), institutions, NUMBER_OF_TYPES, NUMBER_OF_SINKS)                      

87321.88236413943 87315.92287629981 87302.41189990187 87288.93363831285 87275.16384067194 87259.24371865416 87236.54178737669 87219.71011371328 87218.13221199051 87199.20859209834 87176.4738569433 87161.23008735022 87148.7721688704 87126.43526872323 87103.09232279604 87088.31293078384 87080.7270259926 87062.67830595061 87045.72755135762 87030.58174104361 87022.42514396891 87005.68438837837 86997.23030593888 86978.6532287713 86960.02129714421 86943.83798191407 86923.24298124532 86902.37367052666 86882.1368781914 86872.1376326136 86852.9792202933 86837.83892858594 86817.00156626033 86800.77513065485 86788.82433543245 86769.02591813201 86768.46945825945 86753.12640408803 86731.07714150335 86713.25542444513 86693.41829036237 86675.38778527238 86657.42641676382 86634.89201495945 86631.90256105036 86614.30074103325 86593.06699712243 86572.40410101903 86546.87218392026 86528.72116542187 86527.37301024536 86509.68180236407 86489.65540191998 86468.33767509174 86444.31230860949 86427.87351713322

(65921.67410341187, [2.1141975308641965 0.01271420674405749 0.0006598046978094491 0.11655773420479303; 0.25152017689331124 0.012672953639761371 0.0006145046737807324 0.09725880401911995; … ; 0.7919389978213507 0.013071895424836607 0.0009780634343998867 0.14725105728565924; 0.0659648627007682 0.004859129532498635 0.0009626044289872564 0.022774688510419083], [3, 2, 3, 3, 2, 3, 3, 3, 3, 2  …  5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

87321.88236413943 87315.92287629981 87302.41189990187 87288.93363831285 87275.16384067194 87259.24371865416 87236.54178737669 87219.71011371328 87218.13221199051 87199.20859209834 87176.4738569433 87161.23008735022 87148.7721688704 87126.43526872323 87103.09232279604 87088.31293078384 87080.7270259926 87062.67830595061 87045.72755135762 87030.58174104361 87022.42514396891 87005.68438837837 86997.23030593888 86978.6532287713 86960.02129714421 86943.83798191407 86923.24298124532 86902.37367052666 86882.1368781914 86872.1376326136 86852.9792202933 86837.83892858594 86817.00156626033 86800.77513065485 86788.82433543245 86769.02591813201 86768.46945825945 86753.12640408803 86731.07714150335 86713.25542444513 86693.41829036237 86675.38778527238 86657.42641676382 86634.89201495945 86631.90256105036 86614.30074103325 86593.06699712243 86572.40410101903 86546.87218392026 86528.72116542187 86527.37301024536 86509.68180236407 86489.65540191998 86468.33767509174 86444.31230860949 86427.87351713322 86419.54537310508 86399.06878517591 86381.13906101987 86356.82561763641 86336.92777551366 86317.87142143807 86294.67410569849 86274.5292537049 86258.25091139005 86236.89406591574 86224.49111483186 86216.66841426067 86193.28023772589 86170.88867087977 86148.30245326272 86134.06573526846 86111.28468075623 86091.46701928496 86084.78375192439 86066.92391134224 86041.77264503625 86022.46656474094 86001.60217738114 85981.51694197257 85957.40555143723 85956.13047122397 85944.15098519709 85931.4475679812 85904.81267231362 85883.70131696085 85858.03647481045 85851.10462776101 85828.67945987853 85807.49768451664 85787.50849690048 85768.29795374571 85766.24933176205 85742.87382041056 85731.46647483982 85730.26405516636 85707.13854628804 85689.47925725533 85666.20117857026 85651.56004583939 85634.8211723812 85631.66271060343 85621.84328329918 85608.71691013219 85589.10363482969 85585.81419859026 85564.13369971218 85545.53939969964 85530.31004717309 85511.51142079916 85487.37150429023 85461.30339247124 85436.9620815623 85421.30403377229 85396.12643927267 85374.9354386689 85354.57151039329 85330.21262751361 85315.07782326148 85293.47898634488 85268.60118552357 85244.90419284918 85219.15149275819 85197.29593936748 85172.07086658952 85170.99333253149 85169.48069528179 85143.50852018154 85142.30225209976 85118.2101004763 85091.30388105639 85082.4478760319 85081.02830264937 85061.72213127301 85036.076614463 85013.03981115152 84995.4667817997 84971.69264311806 84970.2679939295 84969.37683021481 84967.83089703867 84942.50460026402 84942.49502536462 84915.0429880603 84910.13191004223 84885.13198911096 84867.14359317604 84850.90477935481 84832.81429936565 84810.30443481579 84805.60467096587 84793.25119306709 84765.69476217756 84745.89261127617 84718.59390495053 84694.9925373638 84670.56735810896 84642.98346804416 84616.40434531575 84615.96990244374 84596.26314886229 84571.91470529378 84546.30250218547 84544.78015844969 84517.21142565418 84515.79514414434 84496.82249372329 84472.426626304 84448.36260148915 84447.57363316441 84420.52398152843 84396.5631253232 84395.67125592176 84394.9606467281 84374.14680623337 84372.53486652377 84355.07478519971 84336.26455414036 84311.41763346699 84305.42045473396 84305.20002957823 84285.88390779296 84256.80779932317 84227.62501902661 84204.43620364278 84183.06925881411 84172.24837688555 84151.60806091584 84122.1342393084 84094.62129660048 84067.91659382689 84047.20014978356 84045.74752065807 84044.8571577217 84018.0267921708 83992.54378591524 83991.67710113664 83991.38008313073 83990.96949897465 83973.36800115196 83955.84285520797 83937.97805689262 83907.89452970627 83879.73193368895 83879.45940861871 83878.74099701447 83850.26658486498 83848.57968938893 83843.53827969276 83830.91460371182 83814.71319493481 83813.78439228506 83795.29039192115 83772.49224609653 83756.54949638419 83740.10870875792 83735.56055911737 83729.1391222499 83702.61414486224 83701.82999372427 83698.98316918609 83698.29649028671 83668.5087484885 83667.33828847957 83638.8527507181 83623.75226000496 83621.14667661245 83591.14371812138 83568.07170050143 83542.97613571296 83513.61538620075 83490.39989237113 83477.28556440749 83476.1440372089 83446.52375576399 83419.72580850335 83413.53543259027 83385.95435011112 83358.94083071126 83358.8189112782 83339.97600104351 83312.12099892827 83286.38390960776 83278.81198528793 83247.82357991983 83220.85841577053 83189.29006611808 83160.79596962158 83158.45955612257 83157.57843027211 83152.47332694684 83127.5575606649 83126.98547768728 83122.12537632386 83091.69552333829 83061.12182274892 83034.25893039207 83012.12636388966 83011.49286500689 83010.57046506692 83009.51818514662 82977.83150614337 82976.39531283974 82945.5251090726 82913.63109030202 82880.57804464531 82850.02389029229 82816.78111583373 82813.00710841069 82810.55770115134 82786.52111554965 82758.94340899152 82758.4952613537 82749.93368572246 82723.60676602504 82692.356437131 82689.66742677461 82660.13661057649 82640.50779586578 82640.19005859841 82608.22600989862 82580.3079585238 82572.41632244289 82538.17044681808 82506.11600413425 82475.0554774312 82461.59000531782 82459.41062297211 82458.76158420871 82445.81997926516 82431.2399759486 82427.32556015522 82426.35773979392 82391.53522701748 82367.09986711762 82342.62816678316 82311.68198153297 82310.0751079137 82276.25613852011 82275.53150030409 82242.35079836776 82225.38074761876 82201.51839653144 82170.00563445193 82134.36063749088 82104.3902491825 82103.57419936296 82067.72350879798 82036.07940015575 82002.77829685796 81969.32488397232 81947.88350678701 81947.19397731898 81934.36999624434 81932.89136713342 81917.32319254184 81915.82626260945 81884.63614905729 81882.38532121094 81849.54379617509 81818.74796193956 81810.57172856688 81807.8831908611 81774.1110276163 81773.95391651797 81743.55407197295 81742.16690885476 81711.82017099089 81680.57228701965 81679.14830997336 81675.88164147477 81674.4271310274 81673.61890301836 81670.09315038491 81662.74351487524 81662.09264716484 81657.30506549735 81654.59395458418 81618.73611308046 81612.29675889488 81587.90087319484 81582.95182820392 81577.19853143698 81556.70644482235 81553.59527614726 81525.2342995991 81503.75815418038 81495.66832718719 81493.325457438 81490.57565017461 81480.60764165815 81448.47834153027 81446.72728743104 81410.0634278265 81392.19366442737 81391.44698308024 81355.27685053826 81326.2745240585 81324.43725048758 81292.8497115592 81266.4744705345 81230.53916830644 81228.74119778912 81193.5469953247 81181.10598969829 81176.50287693698 81140.61445233386 81103.5488724664 81070.29150451774 81034.0133967819 81016.22336881662 81012.97533485258 81012.07650032594 80974.98799946 80936.77683642163 80905.65673133747 80905.57757177125 80865.63339570897 80864.20035937616 80830.19729434978 80798.18539376049 80791.28213936079 80788.08523273376 80780.59649262008 80743.32623007135 80708.70208455762 80679.42265797069 80640.79317803153 80611.03468972331 80595.46260329318 80583.19732074713 80541.95483298173 80539.6088228264 80509.05421578129 80488.24285695632 80452.29957953 80450.4844262007 80410.51839738383 80375.7650707948 80340.85587277115 80328.70654694336 80318.40575270858 80282.37643911582 80253.9427948041 80214.80060645302 80214.7045459641 80197.82854278506 80197.63301174874 80195.88209260604 80153.03477153547 80151.70657660256 80149.02510783811 80148.31343709108 80147.35240252194 80112.50511482514 80110.64222226985 80069.23256389413 80039.98115106623 80009.34823528209 79976.87585839801 79949.82851096816 79914.37796980274 79894.74689391167 79856.11377291719 79815.47204238496 79812.68132559613 79773.82694733868 79754.83356496778 79716.91302888274 79716.07912181369 79708.42598790683 79707.26071434146 79703.52314581623 79697.75503920473 79695.17863464693 79676.6167121692 79647.40118712289 79644.05188336661 79641.35601578739 79640.44462434918 79640.40375555099 79600.90548892537 79566.83311053689 79531.9539336338 79488.68191967269 79488.63778152541 79486.29624956306 79484.54694376407 79438.77056866616 79437.63751622579 79399.92717279984 79384.56803139926 79353.31622192568 79332.28011061126 79292.4886738268 79259.07962167921 79258.2972656943 79221.07593676845 79183.47474274394 79177.16407362776 79176.14766501533 79172.20824112427 79133.73279739515 79132.60045434812 79132.03055880476 79120.4057301961 79088.18170862581 79046.12989580438 79045.56969309646 79044.48405969432 79043.68868271066 79012.15063933977 78980.22139832462 78972.58823523199 78970.93761080077 78970.73205306029 78929.76706488046 78889.16691490116 78851.82991844435 78843.29144114332 78841.00407533979 78794.55660302444 78750.22081107585 78709.04610321617 78708.12794209871 78706.57538437465 78661.86067848581 78657.76058325081 78655.62264847112 78619.02423233491 78618.7685394389 78613.47608650426 78611.7670513184 78610.77444456216 78610.6376995324 78565.4501188122 78519.77241619675 78516.6196362305 78485.10441306568 78485.06733248393 78434.75930845394 78434.31288279504 78393.8960581187 78391.64848575638 78390.11463952108 78387.91871481885 78381.42098333003 78380.89157097957 78380.79019304688 78345.63178609907 78342.8439530648 78301.85661326096 78258.97648259788 78212.23435684912 78191.8375070704 78190.32040828482 78183.24378338114 78179.78461101948 78129.94970702437 78127.53945497463 78083.4142312714 78054.66676626557 78004.67570141231 77999.04859884415 77971.36022225759 77925.85139144996 77923.49408071872 77879.42138530266 77828.8655132167 77826.73682732429 77775.86034134058 77772.5594799321 77757.45939569408 77756.56855103112 77755.36174247031 77721.82584895479 77684.02338840152 77681.89953210777 77635.164335821 77626.63643018198 77618.80231662942 77576.63125261413 77574.34450202907 77572.02290795425 77542.2186991085 77494.38662466865 77493.8047714902 77492.79407408809 77488.272089802 77460.05082294352 77442.97154628603 77441.58181806101 77440.17779877223 77392.28447519065 77387.0918625371 77331.44945437266 77297.50289004247 77290.91622507745 77289.79463549046 77280.53030635422 77269.6036036338 77268.88030980996 77242.04616614981 77239.08221665063 77187.97954018276 77183.73394834946 77178.06434841981 77151.92336282153 77151.87381450385 77113.5910797141 77107.89556620436 77102.18229856146 77099.63553604757 77097.0764924223 77045.35416209088 77012.94774808953 77010.62104214518 77007.45298324837 76979.80002503602 76964.3027893955 76962.10808775554 76959.29551455677 76936.36818907768 76880.91922985011 76822.40239044806 76782.75553299418 76732.19927943205 76726.42116306108 76723.44748837664 76712.89673679761 76657.66622986003 76654.89875746839 76597.93164744477 76596.13384928128 76595.02322407158 76594.29521736644 76585.08884064775 76531.44254631399 76529.04841976457 76476.6757581574 76473.47648321126 76451.99023173406 76399.62613410561 76341.20200716684 76337.16380092775 76283.45195617877 76244.61387221071 76243.64036605482 76241.48367266907 76240.06625971128 76181.39689519028 76177.8488414761 76130.43016974523 76126.41420936512 76122.30153820565 76074.64093758968 76072.20593011905 76069.89208134849 76042.09294636633 76040.88293250126 75992.4326667031 75989.93521175771 75988.36705792273 75987.26638252664 75981.27031099264 75954.78973495288 75953.5071325018 75950.40561132615 75947.6954698264 75944.97920429357 75935.08517926652 75882.01341115337 75878.05328198893 75816.53267455113 75756.78451255773 75717.00701320491 75657.94313441217 75655.64973291047 75601.38625134557 75573.13379053718 75558.9293138484 75556.10816376033 75550.97171547 75492.43586482137 75489.52763554065 75486.76048532757 75482.39231396705 75462.95530057099 75455.64579943319 75454.86371700079 75451.2114991589 75403.64253664565 75372.62221532248 75312.4563604792 75305.72230410861 75283.62915732182 75283.08106607986 75282.88241090607 75280.99694338854 75278.0636665388 75235.13810374141 75230.84585051447 75230.52575426514 75162.05646661087 75161.57507221603 75158.56362585789 75148.79567880032 75127.92337732852 75106.02556231576 75104.94984197083 75100.93424964871 75097.620484401 75094.54493375856 75033.53920146203 74974.64791347065 74947.23206775075 74938.8746773958 74917.7089065838 74863.30801476001 74853.44772475581 74833.70064983623 74832.35642121852 74808.37002454097 74744.56766665309 74741.50266676875 74738.27986202443 74734.26710895167 74673.47661434235 74608.98441381303 74581.22369266335 74523.20903071428 74452.5724449192 74449.70480286775 74400.03709650222 74382.18491367664 74382.15951457607 74377.38848206487 74375.96920762441 74370.02068239395 74356.4495131945 74353.23016695573 74353.11767375402 74284.59058395396 74278.88362098302 74278.81665311885 74256.07667463378 74252.51673304735 74250.35350656947 74221.76907881154 74218.38348904627 74216.60677627254 74214.97467446969 74197.47566605642 74196.0492022948 74193.86545236755 74189.35495031021 74187.63901357075 74184.73423490618 74179.79001062551 74175.32425740738 74173.76349591833 74136.0640501994 74126.25318067367 74123.21898944188 74122.64663976635 74121.13395344191 74120.92645822203 74112.91574876219 74084.18398571188 74080.86853450607 74054.32185704948 74049.25954385521 74043.75756152965 74037.84965635817 74032.71965844643 74020.84047472096 74002.99815821438 73935.59181094615 73870.83096828166 73866.79943570541 73863.39284879423 73845.08503180085 73843.96868903813 73841.14288018069 73764.92649905021 73761.44194035973 73734.07492666684 73733.23352558745 73730.86751299798 73701.33102351718 73697.27495804185 73695.44354597844 73655.1465670004 73649.84852760166 73647.3221411317 73627.44849238895 73623.39060130752 73617.54769706771 73616.71763406786 73606.06158635061 73534.95364791476 73525.14272289176 73522.50027836823 73456.51553495796 73452.4241111347 73421.37947391906 73416.8042875796 73414.73715802608 73408.75741231983 73400.66312837545 73361.17288708832 73357.6686851099 73332.07662021516 73325.68373570059 73303.01702735525 73236.08798007498 73224.7636565627 73149.18507895307 73067.93682906717 73066.30896153967 73062.20621501142 73032.23471951073 73022.18180427438 72954.26543336625 72917.28913177636 72860.18923408016 72859.6136767643 72855.93534640183 72853.64008446051 72840.35166349667 72835.93577311261 72835.57158054598 72829.15868095461 72827.61394419856 72772.54900243474 72761.52885362615 72761.00698889856 72757.65349510842 72747.57652282254 72742.8093742151 72671.06781528285 72662.92884009669 72654.28833550196 72652.31232494228 72646.04618640977 72641.86814097875 72640.56001851517 72633.25028153595 72547.91094499746 72544.67917374698 72536.74648893194 72516.38346782564 72428.23467117053 72387.19864023809 72347.33744210181 72342.40350774619 72339.23213836148 72327.45028878404 72321.64417373008 72276.81098905946 72261.40277050275 72254.45738468926 72249.94066304449 72245.9706955117 72245.48935368181 72207.30862329146 72205.22177062875 72203.66045955103 72200.59785053194 72197.90969893726 72177.27408405753 72172.619095778 72152.07174220427 72112.85003999893 72112.04188876305 72107.10778413333 72103.70456228041 72103.16372258858 72098.99289573685 72007.1103724453 71987.80151499849 71979.24963725325 71974.33715969819 71972.24324090249 71971.31564296412 71965.30774664736 71962.47989105467 71957.91045205462 71957.46442401707 71957.43334115364 71950.72320158068 71851.86450944986 71849.61693003606 71847.10832143719 71835.53538391416 71830.21521839102 71815.62110255068 71813.21387547387 71810.14348867144 71802.51842087132 71717.90858293335 71715.96737438042 71711.2741623575 71707.50947809138 71675.43858256376 71668.25988148281 71667.11397856624 71655.50449957482 71624.49623601623 71615.98536292631 71539.94383558161 71529.69472592152 71528.74132574788 71524.04891603107 71520.41429034472 71499.63143720846 71491.25713628466 71488.35888526865 71391.30984028162 71386.57448602258 71282.55085715956 71240.60968083514 71235.72264618643 71233.06575007449 71228.82391074508 71199.12113942832 71189.97894694665 71186.94770258068 71185.90943528978 71183.86580467707 71174.54760954871 71169.63083822317 71153.3463070049 71145.1831736072 71140.39717708837 71114.7873241013 71073.70841226267 71058.85774181143 71054.29295671103 71040.66237905179 71010.1803607394 71007.45003132882 71004.05255790983 71001.55787069992 70999.27905688634 70994.56858660435 70950.78453031344 70946.30436361002 70945.05581321791 70902.7441490198 70885.94741880283 70775.6554165431 70772.34755637462 70762.74811074755 70725.4932041868 70720.27479165907 70716.68991860592 70705.02302403741 70704.81927674756 70689.06722713463 70683.98629348648 70667.21891724128 70648.06331422213 70616.81482698653 70608.69982259771 70554.16336645676 70516.24674296995 70511.38938374727 70413.30138971521 70408.71088633571 70397.89818313037 70392.53713856978 70388.32486674364 70366.20047751055 70363.53942594423 70358.65657717035 70346.83058632896 70346.22368592136 70341.29254008317 70337.74490841723 70335.53678984655 70243.17476477877 70238.88935169118 70192.7075708603 70173.14357815665 70160.95073911564 70159.56878260669 70143.94206509276 70136.11901020253 70131.3653334902 70131.3311343647 70125.15478421215 70124.46642714729 70120.08531258447 70074.40436750771 70063.76789846962 70059.06301339908 70057.43386251942 70056.39521493822 70009.24027510402 70004.24298401357 69991.70219191405 69978.51849529601 69976.69173238844 69970.95898683455 69970.8168848648 69968.81062358756 69961.10372462962 69955.38637171043 69951.9828427381 69932.51757521418 69919.95360466723 69847.5324258245 69845.88042733609 69845.28916723955 69839.82072211687 69827.87073525049 69825.23495676272 69795.78758853477 69786.97667130099 69775.55880866862 69757.9263911183 69754.56275493794 69726.39176781008 69726.06804913809 69689.70109688658 69688.03491564897 69686.05125668929 69680.84995849353 69679.83864023564 69659.30345722835 69658.54109086888 69656.53153482739 69651.74056765757 69622.60668095488 69601.22012544882 69537.05049778211 69531.8191976189 69468.58507733345 69468.23504404371 69466.45556100368 69465.14134248567 69453.84668922043 69452.97785205707 69447.57215534175 69439.83100175811 69434.3451853567 69434.15294076134 69433.2173065345 69430.44563491801 69415.48290941423 69412.91191935174 69407.42040293785 69402.26760565018 69396.4066043061 69369.71979354207 69367.31667552693 69363.21500683558 69338.88985467488 69333.13919417714 69279.17870737505 69278.44182839464 69277.76095851933 69277.52025179206 69214.2109653718 69172.4312265199 69165.09556558204 69163.9723330047 69163.04268101933 69158.54878872701 69099.70704126445 69095.10245246164 69032.972290448 69032.33490863552 69012.29157489422 69006.2328525062 68990.62109142805 68985.84863679041 68985.49019734227 68953.79340514328 68925.23633726459 68921.54305681611 68909.45576476154 68897.52899626172 68891.45223280047 68883.33900624105 68868.99423579827 68863.69824294958 68853.26104710714 68823.01626810222 68783.8749164729 68783.80604851116 68760.45471800133 68759.20208797159 68699.74128248307 68676.51811721962 68673.69217543076 68668.63467689912 68650.81969484824 68633.9936804623 68633.69161171386 68561.11421199958 68559.63307621634 68552.37173489857 68518.4775281209 68517.59857818577 68485.48033428099 68471.53947028489 68414.16913101354 68407.05102468569 68404.09909863604 68351.29197117391 68343.76126430287 68343.58129491085 68339.82517376887 68311.79609757093 68309.47370073818 68308.25405354614 68307.30828779284 68299.83474284013 68298.94336186448 68297.83491430279 68290.81714857256 68288.73645718035 68281.16350190312 68277.59585612408 68275.10700220062 68227.39649549278 68219.62282574411 68217.67000080859 68213.63208124216 68209.3422789826 68197.76975224102 68186.61550530352 68132.85983197823 68127.72546269809 68123.60614020491 68121.04457992908 68053.83191979084 68052.78273528843 68023.1910619626 68021.93223169156 67999.09041942008 67987.26779913822 67986.78414373158 67980.35448554964 67973.51768425749 67968.42396359307 67964.83863093074 67960.53874860922 67957.54863913365 67956.11189543249 67950.65394771534 67946.99297900675 67863.40348263552 67862.4662440871 67858.20777925692 67852.76069743393 67763.5891780323 67738.44465631389 67733.58053693808 67733.28254689161 67722.80749549842 67680.45218217233 67672.82412762404 67668.44601033484 67667.20511194911 67592.78883256075 67586.22563993765 67578.48149726164 67577.21116348525 67572.1394145806 67570.11850857461 67556.93458536293 67539.77299666886 67531.9415973504 67531.92886547954 67524.03724120566 67494.11108816949 67493.59043955128 67484.69091793495 67484.51344581824 67480.53173223374 67441.02951751252 67435.70356290649 67428.1421122654 67427.6470568569 67420.03432690847 67418.76907853861 67411.18974486826 67409.29753670542 67399.11983007194 67392.5415895366 67391.21134953857 67378.02829918853 67367.78651970999 67367.2476611906 67364.37438789912 67362.4997789838 67347.5192378927 67347.41809204189 67322.24651767522 67322.18744604288 67319.11960008641 67316.16146415994 67307.40076039522 67301.30630022772 67301.2878176367 67301.25951745787 67295.07212449342 67284.12731128406 67273.18200523288 67256.06551712142 67255.13422491257 67255.06966757042 67255.01583957343 67252.29913342178 67240.35462607161 67191.91746004767 67185.6968967617 67183.45124747517 67179.5718688039 67177.01621600626 67169.11164945943 67159.94966436266 67123.22185521774 67119.75833986285 67111.74257755354 67109.03572673684 67100.4905622904 67096.20768734036 67091.28716090413 67082.35628116615 67080.25038023524 67074.63095982678 67067.37926547203 67066.66157710792 67065.74481325428 67060.48284055124 67055.03953335651 67024.58267795155 67015.93728472177 67012.38069745955 67009.69247593991 67009.29307547 67008.36395149674 67007.06102508653 67004.32431692227 67000.84561407533 66994.02840387775 66993.47062575739 66991.33735284106 66936.28039786412 66928.65589287794 66894.63311690913 66858.51039693048 66835.8797286193 66835.68046771198 66835.5770861616 66827.29626845494 66820.07951974131 66813.9208289577 66798.41515330726 66796.59099609459 66795.83364337486 66795.6591842478 66794.86380148133 66786.3963122184 66776.33310470001 66767.04373338999 66764.39864537977 66764.16153685031 66762.28762096264 66758.38029291008 66756.46522109483 66753.56353191509 66742.7616737926 66741.60051699009 66737.46472431446 66727.72644571756 66720.66380724112 66712.281334078 66711.85553197174 66705.9633249983 66705.67901713253 66705.11795036095 66704.43329795024 66700.32740960516 66696.36077261285 66696.33679085503 66696.01376669007 66695.83201192078 66695.72688077707 66694.09081556358 66691.44620475863 66690.84922861386 66674.76606347844 66674.7035046138 66671.72534431954 66668.4050235943 66668.28687095678 66666.18665701206 66665.83962951128 66664.48594097495 66664.05997411252 66661.9872461413 66658.39542921812 66638.73854880451 66636.26390216275 66635.9373395864 66630.76716161585 66628.62458399829 66627.83947633393 66619.98353408107 66615.46447378342 66612.80225422872 66607.37767564405 66601.7685947525 66599.63297549404 66593.93084715915 66593.40715170588 66593.21715097256 66582.81714693508 66582.21030992466 66573.35205928405 66566.6029797461 66558.0990608149 66539.12841169398 66525.35413988195 66517.76725957719 66516.05009946701 66510.3068676087 66506.48090588186 66506.46435493923 66505.53545891277 66487.76114512663 66476.39834822337 66476.0268315444 66474.43989179349 66473.31183063376 66471.05694160845 66452.56065035891 66451.39244534221 66450.62936815777 66437.37543127264 66436.09843585685 66411.52474298242 66410.45863406632 66394.31140571262 66392.49295391477 66390.94146587081 66389.24061951553 66384.39276025983 66384.35675648475 66383.90675470387 66381.19723479544 66377.40388924848 66376.94438178587 66371.14962982606 66368.20466060472 66365.62030129279 66362.66773850078 66362.17871604749 66359.03920879247 66356.12558288443 66355.54693768344 66341.48501015877 66341.39179073226 66309.32210864713 66295.78266774425 66294.92127065183 66294.90906329424 66288.83792056414 66284.146522021 66282.14226512377 66241.45944942074 66232.31993879918 66222.17949010979 66173.59891477859 66165.72249882306 66163.91725403717 66160.84547154822 66160.699730412 66159.36122384894 66152.07638362034 66146.40500258014 66145.80645064682 66145.09734668219 66144.55797922061 66142.8476201013 66142.6011728561 66140.28887904677 66137.2732809367 66135.81165859314 66135.521135845 66135.04110522346 66130.2725252417 66129.74570663924 66129.61886402687 66127.95935120032 66116.64645711517 66115.26329090013 66115.20141181374 66113.27613204297 66112.93747154681 66112.5909616763 66111.65503685843 66108.02438160435 66101.19277828796 66094.29843029568 66093.98109336164 66093.87231163634 66093.53282647459 66093.46590875753 66092.43864144907 66091.11566763607 66089.98839535793 66088.95014960655 66086.95411115723 66083.30674145996 66083.05462954135 66082.21218355634 66081.12161022049 66080.79667117143 66079.38334347402 66078.08751683167 66075.91300329928 66075.30860535438 66074.07849341222 66070.86479385132 66069.60803312257 66069.60469828338 66068.62993040892 66068.00407012584 66066.39619385677 66066.1279738442 66062.94889820428 66062.53449816935 66061.87451576977 66061.69816938318 66061.19911340719 66060.65863946412 66059.53817570778 66052.6518457847 66050.89885605151 66049.30206463372 66049.22527506542 66046.59834381062 66045.19773663313 66044.59790738451 66039.84596714035 66036.9283133393 66036.58718272079 66034.55760916375 66033.43082000567 66033.26742804327 66032.88274642006 66031.33331020985 66031.23554280105 66031.1572773835 66031.10596164377 66031.02649974146 66030.27787905544 66029.8426819392 1000 66028.59768466279 66027.42152060493 66027.14108129921 66024.16878226312 66021.81896278319 66021.77191890446 66020.329775367 66020.29006999283 66019.52365145722 1000 66018.9008243071 66015.84579286104 66015.84005034932 66015.70193289155 66014.68644436271 66013.59264235997 66010.34415120905 66010.10165994005 66009.51779845066 66009.20166364517 66008.73977572804 66008.44725582079 66007.7197832139 66007.36988583402 66000.52789769154 66000.50973288665 65984.77342756806 65983.31696070317 65982.8884685333 65978.9592766507 65977.4462676951 65973.49088599393 65972.3156794736 65972.1402524032 65970.71052412716 65968.79716016707 65968.17349186832 65963.77891725015 65963.6511561842 65963.33267825482 65963.2237670096 65952.6268262506 65948.82953756051 65948.6078137393 65948.59831832863 65947.8143243882 65947.34963073776 65946.72313203244 65946.32849517897 65943.60982353195 65942.7290427962 65942.6747721438 65940.31539753858 65940.2168064008 65939.79483350179 65938.04234327604 65936.87814845455 65935.82621289493 65935.3522714414 65931.01153311944 65930.10029738407 65928.58497015979 1000 2000 65922.81863821809 65922.60104027521 65922.01397134637 1000 2000 3000 65921.77281752179 65921.73495634418 65921.67410341187 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000 51000 52000 53000 54000 55000 56000 57000 58000 59000 60000 61000 62000 63000 64000 65000 66000 67000 68000 69000 70000 71000 72000 73000 74000 75000 76000 77000 78000 79000 80000 81000 82000 83000 84000 85000 86000 87000 88000 89000 90000 91000 92000 93000 94000 95000 96000 97000 98000 99000 100000 13683.349804 seconds (1.80 M allocations: 3.993 GiB, 0.00% gc time, 0.00% compilation time)
(65921.67410341187, [2.1141975308641965 0.01271420674405749 0.0006598046978094491 0.11655773420479303; 0.25152017689331124 0.012672953639761371 0.0006145046737807324 0.09725880401911995; … ; 0.7919389978213507 0.013071895424836607 0.0009780634343998867 0.14725105728565924; 0.0659648627007682 0.004859129532498635 0.0009626044289872564 0.022774688510419083], [3, 2, 3, 3, 2, 3, 3, 3, 3, 2  …  5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [ ]:
function doit(sample, academic_institutions, asink, gsink, psink, tsink, all_institutions, num, numsink)
    # some initial states
    current_allocation = Array{Int64}(undef, length(all_institutions))
    cur_objective = Inf
    best_mat = nothing
    cursor = 1
    for inst in academic_institutions
        current_allocation[cursor] = 1
        cursor += 1
    end
    # the sinks must stay in fixed types
    # this was built to support more sinks, but by default we only use one
    # change the "current_allocation[cursor] = ..." lines to group sinks together
    for key in asink # other academic
        current_allocation[cursor] = num + min(1, numsink)
        cursor += 1
    end
    for key in gsink # public sector
        current_allocation[cursor] = num + min(2, numsink)
        cursor += 1
    end
    for key in psink # private sector
        current_allocation[cursor] = num + min(3, numsink)
        cursor += 1
    end
    for key in tsink # assistant professor at teaching universities
        current_allocation[cursor] = num + min(4, numsink)
        cursor += 1
    end
    blankcount = 0

    # BEGIN MONTE CARLO REALLOCATION ROUTINE
    while true
        # attempt to reallocate academic institutions to a random spot
        temp_allocation = copy(current_allocation)
        k = rand(1:length(academic_institutions))
        @inbounds temp_allocation[k] = rand(delete!(Set(1:num), temp_allocation[k]))
        # check if the new assignment is better
        test_objective, estimated_means = bucket_estimate(temp_allocation, sample, num, numsink)
        if test_objective < cur_objective
            print(test_objective, " ")
            blankcount = 0
            cur_objective = test_objective
            best_mat = estimated_means
            current_allocation = temp_allocation
        else
            blankcount += 1
            if blankcount % 1000 == 0
                print(blankcount, " ")
            end
        end
        if blankcount == 100000
            return cur_objective, best_mat, current_allocation
        end
    end
end



In [ ]:
state = copy(Random.default_rng()) #this reaset the random seed so both tests have the same set of "random numbers" 
Random.seed!(1234)

In [ ]:
@time est_obj, est_mat, est_alloc = doit(out, collect(academic), collect(acd_sink), collect(gov_sink), collect(pri_sink), collect(tch_sink), institutions, NUMBER_OF_TYPES, NUMBER_OF_SINKS)

### This is the output that the above line would give
87167.71403596654 87149.61885406218 87148.22380765609 87137.3598778919 87121.08450287407 87107.49513358274 87089.95521100592 87067.22995523066 87052.42110819285 87033.77503675935 87014.16041146395 86990.95315372797 86981.13212814093 86958.77089993196 86944.27284229794 86935.87397400678 86917.47202784251 86916.74157582349 86897.942167008 86882.51817746961 86873.79466863033 86873.2255956182 86854.54367711033 86843.87163316063 86827.37846181304 86807.8140260719 86802.78642604398 86786.98998096393 86775.48052296237 86756.62538460981 86737.6565426398 86723.53625220232 86719.0358606693 86711.3272902198 86693.42959895938 86676.7811185076 86654.84144016127 86636.91262302686 86624.56779982938 86603.95608740965 86581.21764476171 86562.2123948964 86542.25617678998 86523.72551719293 86507.42913447478 86488.53174148615 86471.28009163613 86470.28453385635 86453.39969779807 86449.97760778978 86446.62987768483 86427.57282611691 86405.44950754986 86383.13600645104 86360.05754275116 86344.10307452256 86320.55333416148 86319.10074547828 86300.78081766663 86278.03511877089 86255.91857782418 86232.92790463418 86231.41811084808 86223.49173332578 86204.26294299567 86185.96080498993 86164.47679715225 86144.34686904722 86120.5195254958 86100.11652467874 86077.95143727778 86062.62772052718 86058.82515073133 86058.11223829188 86036.0253509285 86012.50491928413 85988.82669877828 85973.12024838332 85957.39600828306 85936.45462461471 85930.01909373583 85911.35882489545 85888.19301704575 85868.69325092186 85847.9718765884 85823.89010478681 85799.13486552275 85775.45404618033 85774.23200159606 85763.123667893 85753.87042197675 85728.79015925525 85706.66020330759 85682.66871688623 85677.16397827596 85653.06465176311 85629.82027499902 85628.86781360637 85604.5548980143 85582.93637571349 85563.00914124158 85543.80168923474 85519.21735303767 85508.70690338955 85507.58680781185 85483.77008354448 85466.43973471073 85442.49995324542 85426.42067673136 85408.27310740697 85402.1175365539 85391.02973982232 85379.92963322099 85361.0062355132 85355.04860045738 85332.15835034962 85312.7380270489 85297.7488903285 85292.35422794214 85267.16796851566 85242.82859387898 85217.46209313608 85203.39225107034 85177.12545998767 85176.7870912172 85156.20926661488 85133.06242448997 85126.2109300678 85103.58580122956 85077.77158887523 85053.09758626482 85027.11014513765 85004.99831114085 84997.28306014171 84970.46192089458 84946.74033634509 84921.67592050991 84911.852391156 84891.38958791652 84871.79085001812 84852.88622584227 84829.36452162777 84811.96706673999 84787.38540602036 84761.52049472158 84758.22452163088 84753.03892735996 84727.28060026935 84700.87423693291 84695.79715730867 84669.8568581013 84647.04158779424 84629.04481181609 84606.260707402 84605.5083138072 84578.53123525668 84566.28418668175 84539.79639717996 84519.85097051282 84491.65119311804 84467.24300915224 84444.87653301487 84416.42145004222 84401.82134325331 84374.3272006225 84347.7067807016 84330.30999181622 84303.99933220081 84276.41943029783 84248.93174884429 84223.03062218022 84200.10852094131 84199.4707165633 84174.82580360076 84173.85531137718 84146.3348395354 84128.73011486737 84123.76539719992 84102.6144558432 84085.03092905869 84057.15893744535 84033.74954606511 84005.67353005311 83980.01263594512 83955.39915794235 83932.24195651895 83910.3276297308 83900.51645803779 83879.43364683898 83852.62958642124 83823.95690436049 83798.7366519996 83771.00789992248 83749.87753024005 83720.62541438876 83719.90185445028 83691.77354934927 83665.33009794973 83643.39905490288 83624.9293861102 83609.3241454376 83608.69198657945 83579.28106415551 83578.42500098677 83564.12024249756 83553.62098350374 83532.93766053191 83524.43371585393 83511.77884380914 83494.07971107976 83493.75252709181 83471.52447784165 83458.22906458183 83431.23897913707 83415.70030283011 83409.71827791125 83383.2375429431 83382.55292438657 83376.82434373863 83347.36072957783 83316.85796515188 83290.39132331235 83266.7396571635 83255.04953167272 83225.49660109801 83209.89735265508 83184.30343608938 83182.30840430922 83150.72422973378 83127.39879889667 83101.57748875639 83072.29854797687 83071.17031701037 83056.86109065848 83035.58235670268 83035.11461224114 83034.51398385795 83033.832588197 83005.7646873461 82997.86374633522 82969.74764808267 82941.48714144852 82917.20321084428 82896.76213213471 82896.15362504561 82867.76382733404 82867.20116048784 82841.19151394516 82840.94589880004 82808.2630157521 82807.301837701 82806.89315518684 82775.47233259236 82746.37601054572 82745.10333986602 82712.08814033931 82688.28319243946 82687.25200089884 82659.57616752625 82656.24336731424 82623.86622229694 82591.36284751199 82590.81602313202 82572.01127718395 82551.51271843073 82528.09196841644 82527.47207819043 82497.54685254139 82466.73144094515 82434.36369381136 82409.50421752616 82409.06585916114 82375.86767098158 82341.75170408621 82312.48383688636 82283.27144504001 82282.36548407833 82281.8804017306 82250.47520026824 82219.1510733262 82201.95671856105 82168.69553770073 82167.9143438016 82133.22208519849 82101.37800648573 82099.29750870114 82096.38327407966 82069.45642113202 82035.37665589554 82001.64657078864 81969.7201508475 81939.87697790703 81906.30504121268 81904.85469642388 81874.601670498 81842.2022399459 81808.61822740256 81801.32984946352 81772.0120540891 81764.12847013508 81746.75758613292 81745.46531936753 81712.62325564628 81710.50420122164 81675.68362524659 81643.76450744335 81643.16836093797 81641.4997229855 81606.00655266586 81605.3203411238 81580.71163877835 81580.07429503724 81580.05352380521 81578.85953245158 81546.6048847676 81511.51712143958 81479.10129012981 81476.60648663204 81470.12431944847 81469.51131748322 81467.61146855965 81437.97525018107 81406.31914520595 81369.98717895494 81368.78489131149 81346.84352819093 81346.47162020933 81328.40869048536 81292.64334144587 81263.96771944246 81228.48779342615 81217.85436095535 81181.7226812422 81146.00323659834 81145.02685765873 81116.97645690675 81093.03472376234 81056.96185854456 81020.84007140945 80989.06207995834 80953.60657409397 80952.05422999628 80950.3897577763 80949.56229674505 80949.45165746877 80939.89810809433 80935.46065331888 80921.97515179 80921.14291778074 80918.92024233291 80889.45818915898 80854.22292719544 80821.73244768009 80819.39812458506 80794.34539783251 80767.54879298926 80747.42251593166 80747.26225211669 80724.23608562564 80706.32965465085 80705.9341871948 80696.09940059166 80656.20762278003 80626.68610232887 80606.37436496346 80594.00826286754 80553.6946108771 80520.38877896205 80519.53002712085 80484.62581899302 80482.69474870063 80452.02930044908 80412.46543048644 80375.3891822471 80366.38055797822 80365.00120815258 80329.65282138411 80328.53275853884 80324.00829698572 80300.65513889903 80260.43694232195 80220.44366009349 80185.20244367947 80184.7545193642 80184.57117409572 80163.47853408167 80126.03047867675 80122.95068231008 80083.19565088637 80048.38806134253 80043.11854028798 80005.27628399542 79972.70249029304 79970.43994076803 79968.5030177153 79930.11929659468 79899.28872997171 79873.12011695432 79869.56713304365 79868.69797718283 79827.98666907047 79794.58321496187 79792.96035767556 79791.11341253236 79747.83003110663 79746.71108752489 79734.33636106378 79713.81690170303 79706.9250508427 79666.84659240961 79636.90598703823 79635.66218802157 79629.79945336762 79614.60138949259 79578.8656001348 79535.1459625492 79533.37580454253 79491.36086545375 79451.06658117793 79450.31276429082 79449.6591425425 79448.72210534106 79448.03844018147 79424.89716084914 79382.92742985295 79354.26235711276 79354.05392161744 79352.45015215286 79349.96712999007 79318.81794060492 79276.79456420512 79274.4649415061 79272.86687101127 79227.33916494783 79226.19527672025 79180.48974597106 79145.33799466377 79103.9038405545 79085.93642391029 79040.78422192624 79001.48471874117 78998.6876173976 78997.83875833756 78996.46601784334 78996.3006623107 78995.91515296094 78954.89154950873 78954.1227626199 78914.39121190405 78905.30790033717 78905.23578573586 78877.5376803856 78835.20944886185 78797.95959627576 78754.6955611955 78711.89119950822 78707.17571589146 78660.30424985505 78658.25108053636 78656.1074809516 78656.0725353308 78628.8420644258 78627.71378778489 78627.27636424151 78583.67731825782 78550.0005792197 78504.67405089871 78489.79815222838 78489.5963830036 78448.58086460718 78402.59495008316 78389.7975118163 78347.14820452796 78322.50605690268 78322.01310231867 78273.30581384379 78273.1765221043 78269.14264634863 78222.36626706469 78219.53973172937 78176.73417030658 78176.47326929304 78169.02965730829 78152.68756243541 78152.4996763453 78151.74618718498 78145.11465042902 78110.33898810555 78072.31126140502 78035.83758056765 78034.02205302134 78033.74625544777 78031.79033916564 77987.56611871826 77945.37760793102 77943.31411505608 77891.9676374851 77886.82857598913 77837.68716788029 77785.91676166252 77745.57498301449 77739.2976925099 77689.563686873 77688.78113527375 77687.42782265817 77644.47205559682 77641.89167254575 77641.33447808924 77641.22160041668 77598.69004508332 77597.08379754174 77597.08356554018 77577.07861781714 77532.90007682425 77532.8443103482 77529.96973737994 77525.55981076401 77479.53939128465 77446.04195171037 77434.40198950471 77388.71789787618 77387.8902014261 77387.03260387201 77386.08858331632 77385.44231604233 77340.74450695362 77298.21188876092 77243.53668329945 77228.78710220329 77181.2225357782 77180.25569562917 77178.5330615377 77178.12410783661 77175.73971012117 77123.64106624249 77112.47162969763 77102.64379981757 77100.0893755922 77096.0737433437 77040.07092023564 77039.0622657533 77037.9117094408 77007.72734481299 76956.32232171745 76949.49544609184 76897.8248601771 76882.56036953392 76882.09991625296 76880.230366896 76876.49916571895 76876.05694207504 76875.58310820113 76857.09180989079 76854.94844233656 76840.84268374812 76839.35277402533 76835.37103100953 76818.51707834104 76815.06741337001 76813.870744453 76792.96843455316 76791.81871062967 76790.06759449525 76754.15512380034 76753.125570011 76744.55001879344 76743.92916541653 76714.89854621216 76671.11631595244 76637.74971618367 76602.65440446371 76601.50034345294 76585.63976087018 76582.05735726013 76538.51249665343 76534.9466508297 76534.11754583461 76533.36148936802 76530.43222475413 76525.52602041309 76511.93306572587 76511.14996910655 76487.03108703713 76438.09755372451 76437.03122255302 76427.11621704312 76390.54060615039 76389.19717055585 76384.1674447534 76377.62430718089 76377.15694596067 76357.90994237163 76357.7622353188 76352.64310431959 76291.64783066108 76290.00991411797 76270.1489949741 76269.6716641967 76269.47571305132 76264.20291456535 76207.94424673473 76201.75972438105 76145.8280615389 76141.47736129716 76126.20489153023 76125.80479636467 76122.30257358478 76121.07732667336 76118.60513137247 76117.35105501622 76116.78885078723 76112.18224681747 76110.93978608842 76109.85507261484 76053.90448930982 75991.3554781675 75942.38330005144 75927.71944017966 75927.68889022172 75923.33371219586 75914.77453009524 75912.25917911823 75900.14141968887 75842.21329909169 75779.97367581615 75723.35113594522 75689.50735030485 75689.00240594805 75687.69692035696 75686.34893591588 75676.12469879992 75669.2783088293 75667.72894359514 75643.44291563616 75638.78335663918 75579.40251469864 75576.83691485447 75573.48849042178 75555.40290986664 75550.00782710727 75527.80180780341 75526.98272045417 75467.25202497125 75463.89322566966 75409.21662876953 75408.88387351458 75342.26336780516 75338.21209649881 75335.83617071403 75334.23620446886 75267.24972514373 75208.02448967625 75205.31223363294 75200.57202071778 75199.61986774938 75198.43493985878 75196.62852880446 75193.04301023005 75190.18992399631 75159.99669729442 75138.94895708929 75109.34967507033 75103.1331013997 75098.38234976167 75048.81598353412 75025.1722956521 75024.65972203288 74957.18641705834 74920.91553357126 74919.40873872634 74875.37963396599 74867.3799650937 74862.06646641645 74859.04428620538 74852.37975317649 74849.04966765924 74785.37789264161 74775.46284666481 74772.23839619584 74736.94436917614 74735.19997123502 74675.18825753064 74673.46415737622 74669.68448651729 74604.5283931722 74541.16036317323 74540.8364389864 74537.9161986723 74534.962304261 74522.3536318142 74522.19184908642 74500.25495791428 74491.01028006557 74486.09873644562 74485.03869457514 74471.76611368063 74469.89167284082 74412.84574562867 74408.59727279154 74390.07770513878 74387.74121699297 74322.67415781757 74319.4933762313 74296.06486228958 74294.9431063713 74246.48297452171 74225.65374187811 74223.20305457395 74222.8973315545 74204.93507516304 74136.19986029493 74133.6809387133 74060.02842902242 74058.51739445537 74055.6575193387 74055.27334133384 74052.76079747747 74052.1296744281 74048.7446846814 74042.6668389537 74042.66320613821 74015.06823810842 74009.90845806163 74009.08704931801 74008.69496031338 73994.36565671444 73993.36238582923 73991.39102049262 73988.81562893449 73969.75418879949 73941.84693325704 73935.45305922799 73913.46519831615 73909.72782300302 73908.50447927028 73903.79277250521 73838.29068550168 73761.69214814853 73753.45835551356 73753.16636618008 73751.06867160035 73748.46187480041 73747.74710544119 73745.08924384718 73744.46519655742 73739.50057674627 73729.0928664609 73726.99808484738 73721.91021367963 73721.2691313403 73715.84072763838 73713.910549784 73700.76904745992 73683.89204355559 73682.82237807158 73610.7762424235 73599.08399479781 73534.57200235219 73533.71747859613 73533.42697469855 73528.38832692575 73521.10628919031 73519.86089129385 73513.2597783443 73507.76151263212 73502.27425631104 73499.48205073709 73464.40589715687 73442.00483375299 73437.94063796292 73371.03145899606 73351.43865771734 73340.14022193594 73309.41246089594 73308.69401322103 73306.03867105056 73301.05875590828 73300.95644231295 73275.56271219709 73271.70997718888 73258.10548496117 73257.473399279 73191.31403923777 73187.56313519708 73165.63957734598 73165.57576253758 73124.48793985424 73112.89523418926 73111.12157800136 73109.65664385707 73102.96716564205 73094.65898623114 73094.23165742397 73092.34329005906 73090.05690366865 73087.88576567463 73072.38363530682 72998.62072958228 72953.12250958676 72952.72807469577 72946.2396315106 72858.47468027144 72855.8525268395 72853.26800220706 72844.80160657382 72843.89501255969 72841.40662923777 72821.34076812549 72819.93267211205 72814.4250272883 72814.39900417061 72811.58375961107 72796.47006151003 72795.5184078073 72787.8295204193 72784.94013704949 72784.69281541328 72771.44529611693 72770.37882566922 72767.54429085697 72758.97261599245 72758.2395168408 72754.6777196157 72700.53934512823 72672.09466452578 72593.12834125746 72591.64923009383 72512.02621993421 72510.70245087246 72509.79177905586 72496.15286906049 72494.39578530972 72486.82435684024 72480.74732504226 72475.07109529823 72465.40473602201 72464.42900753528 72463.46911693575 72460.96834812404 72459.67031488223 72451.86406055593 72440.35516465708 72437.56165562387 72434.57864561943 72431.98879690205 72431.92761406642 72414.86953865174 72407.60761064268 72396.26808156974 72318.4277998456 72279.41780297345 72271.35954219018 72260.05649552072 72258.4843710004 72257.75032759582 72253.85985654853 72207.25108461475 72201.89947415669 72120.20908925538 72085.35094359965 72081.21614193352 72063.05045538799 72059.42353824488 72016.28244169433 72005.15095629361 71997.98138367511 71967.2207372804 71961.54787713978 71949.22347902035 71944.93749164585 71942.36851390793 71940.11982817123 71839.67351734525 71835.8845217403 71828.01208268202 71824.33393816504 71788.95942522898 71787.25811284284 71784.87206845825 71780.05459070971 71763.07344798058 71685.62569055363 71666.18612173929 71648.20517504617 71641.090899901 71639.3890178575 71635.50989471047 71631.59722311689 71631.4651312998 71531.92178154316 71530.15252939322 71530.02110969348 71527.31663335636 71476.20653087263 71411.24804022037 71378.55546124076 71375.47593263163 71371.4076242733 71368.46569647845 71360.69187950762 71352.17112201583 71351.33606860624 71335.87047346715 71327.8346407129 71319.50615511114 71296.95404459999 71193.15539965608 71161.46778382437 71158.4104877191 71156.3462016166 71148.347953868 71134.28202283656 71109.4943039551 71106.4425997149 71075.30274133515 71013.17794330865 71010.91355309082 70899.87030999162 70896.00452049365 70853.83250768937 70843.06256839156 70839.50790742243 70836.23313982096 70832.92078993062 70828.54397293323 70812.07567261271 70803.3044193912 70799.896139303 70791.98399810668 70788.53674994012 70788.25567420284 70783.40961789162 70780.11792320416 70772.69423379445 70765.93670433541 70762.7469571954 70740.5312451535 70734.60714303242 70720.3854747955 70712.26812858213 70711.31885645501 70707.22263423688 70677.3344772791 70668.07639809902 70639.86741549519 70607.57941974862 70603.74374064356 70568.83424600241 70538.77882211423 70535.1034509543 70531.3783742556 70529.67850826622 70506.89328287022 70502.37109871153 70499.10192567033 70497.62774738473 70486.10274052912 70469.09678903011 70465.79759063467 70463.29123509185 70458.41061510413 70458.30071050025 70447.65840496434 70425.7202496264 70349.47461620488 70331.63516780105 70219.38564784364 70173.250817538 70169.45162035424 70143.45602986524 70073.32437784011 70068.81111717132 70067.24501194751 70050.40658903039 69980.75396884145 69976.15370471432 69974.60064230737 69972.21984365121 69967.56056151427 69939.06133747559 69936.67588446567 69890.70747168051 69888.40524213466 69861.44273497096 69843.49245130512 69837.85629265742 69833.45917805817 69832.07045525618 69828.06121448941 69809.23514111832 69806.39400518245 69804.97035648373 69800.33719531518 69796.52807764735 69794.02264663207 69790.46596975636 69783.54561607634 69779.55341935823 69774.35081717368 69774.11336967013 69748.28305957702 69710.95170979359 69641.83478833259 69603.5298063238 69598.72080484577 69587.12247110126 69582.52834859569 69578.8348768177 69545.27250685966 69519.47259442409 69512.72499313817 69508.75504349815 69507.12224353323 69506.98188166192 69503.90398009474 69498.58244497767 69493.68728724169 69492.92256759625 69489.18218820156 69488.2098062019 69482.71812374092 69461.43795861996 69460.59461943235 69440.49304237831 69439.38256355358 69414.27285745108 69413.63255691611 69411.42027577639 69406.3789364394 69402.91096827434 69376.16497153004 69374.92052586308 69371.0566022293 69370.66954472507 69364.9769783633 69353.87370858599 69328.22376963189 69327.07731112515 69326.12173019849 69325.92164949066 69291.44783645435 69271.35632467683 69266.23411206914 69264.22896297365 69263.81852618272 69177.10893239737 69165.86175157146 69151.32417187568 69145.82755077674 69139.93392804312 69139.88069431565 69133.89643242172 69098.09665434061 69092.13880058807 69084.59012195567 69080.6283055683 69074.55965649607 69067.57693653519 69054.92780834714 69022.55253839605 69019.29382727308 69015.62480985487 69014.3148409516 68968.1969411958 68960.46904280993 68956.65165896088 68908.82751574402 68908.01098111196 68902.80066931153 68896.38190326568 68895.35875203782 68891.3825488788 68888.75202029281 68862.87681482204 68848.78613431158 68811.62685266782 68805.30221421822 68804.78082192421 68800.35784312965 68780.14713367958 68717.58952458874 68694.2804116369 68687.42849802975 68683.08241357651 68677.54891364125 68660.31776039267 68604.57238818258 68598.38026862749 68598.20303328645 68594.65487077157 68572.99630330858 68571.29915001354 68518.91270508293 68471.31692149548 68460.30031988904 68457.0754469166 68456.36146222959 68450.05278094021 68438.4846007774 68437.72527322867 68432.35454385151 68425.9191163563 68424.86384862846 68290.23420045868 68289.64347410057 68255.6257505117 68238.68421993474 68237.92926773298 68235.43039679166 68174.52287431629 68164.18466422161 68160.43015012983 68083.53629981678 68079.22558938916 68073.02802973024 68027.32665615046 67992.6532218189 67982.23331008821 67975.51775388436 67950.82702996908 67926.39269491694 67919.61588788702 67919.26753857678 67917.77189020041 67908.98889606341 67825.12939690061 67820.80707977212 67811.48277796447 67804.96701692272 67800.94512679089 67800.11329780893 67792.44065103742 67785.67119964794 67784.1068340844 67783.52239641907 67780.4691319763 67778.40532222514 67770.47492583324 67766.65917426728 67765.83349118527 67755.0808179818 67731.95273686269 67731.30723284364 67730.26163160316 67726.5784390177 67726.1363005064 67724.00944075998 67719.15102437862 67715.38023025503 67680.74963965091 67661.17894948272 67657.44371158897 67638.57046179465 67638.17358441533 67633.52619681714 67632.0218946069 67620.5549762072 67596.0690437576 67581.7227957099 67558.34029157642 67552.51449098093 67548.24013120348 67547.99568746172 67547.31655140681 67537.91663808761 67537.68088557968 67536.638531262 67534.90658520968 67530.75478202397 67527.61454678267 67526.6048664426 67522.91123567324 67512.34210076812 67509.81625600871 67508.0480874164 67500.26532850147 67496.19142487767 67494.65151223193 67486.8939525352 67466.1014314649 67463.77457460697 67456.44971302309 67448.25187412316 67440.84086056193 67435.0930074908 67431.4034178892 67429.20774997748 67424.13288540616 67421.92849977061 67421.55589826498 67416.3285630802 67415.8812169038 67411.70410222 67404.19959006413 67391.48146136347 67388.4734613088 67376.6916591509 67375.39463951951 67327.050439479 67313.01072102076 67310.87721765417 67310.82483213526 67310.69147357014 67307.23229622944 67306.85074216523 67306.5617492245 67255.4156420599 67249.65631628642 67246.0559890093 67243.72164951434 67243.14052358594 67201.58399271464 67200.72368828661 67194.17917215174 67185.93396221615 67183.2131976372 67154.07902751456 67146.65102403147 67139.16631791653 67138.78945292011 67107.08474284351 67103.24178776132 67102.52767772129 67100.3812238902 67096.73636607843 67095.79387365219 67087.26371374469 67081.155248661 67077.15774598096 67031.79248444168 67016.82443845547 67013.81398378377 67009.20890971832 67004.82143926396 66997.53664262267 66996.41269181564 66992.3431404683 66990.90341133234 66941.83700292249 66938.03667762999 66931.75602757726 66931.74386998218 66926.69147564491 66921.68541134374 66913.12817672934 66907.09115336805 66905.61972446072 66905.43726948349 66905.21540071264 66881.13104702294 66879.04809475406 66870.87545797578 66854.97558241677 66849.16812453017 66848.69035199926 66842.95077380684 66842.54037061051 66840.51399211341 66840.42479442523 66832.27412747202 66827.86137244888 66825.12499373077 66816.92393949628 66809.33482160495 66804.72777814027 66800.70233395329 66800.03461732177 66799.73159453682 66794.35506050466 66752.99426560116 66752.3412882237 66744.75068140657 66741.56407669905 66741.47734069856 66712.09543789522 66697.94778665525 66696.565616747 66696.49438016431 66689.91490196668 66684.1758472892 66684.01772441779 66675.7784633123 66661.93878345129 66660.885640425 66655.87452423248 66653.0794676184 66650.63428294618 66650.40238331766 66649.51366600457 66647.80382892142 66647.6596788887 66638.82294677602 66638.45283849142 66638.099302394 66629.0940151145 66628.70227417498 66626.93260313319 66619.82571723375 66619.30992517632 66615.63062178982 66610.40530991426 66571.1375270656 66565.61325431366 66565.58130610436 66563.52091834613 66563.12821159726 66561.94242217869 66561.73668985748 66560.98532739887 66560.96767640345 66560.5601299048 66555.1863847816 66553.4523748752 66549.48355191578 66540.65445783988 66538.96008169625 66537.5413066855 66537.25547949782 66528.2620826575 66521.79494158152 66518.07165268098 66503.64333850604 66502.77985403335 66500.55845244706 66482.95311453166 66482.82975249676 66476.18465918035 66474.88755441332 66472.62534635441 66471.50843545314 66466.05922103555 66465.6768695188 66461.01601204916 66455.18916366335 66452.9304424318 66445.34909676903 66438.39044889805 66429.41643594715 66423.86370598934 66422.06257689037 66421.31437537997 66420.7101397019 66411.37407166514 66409.74062124953 66409.10232364596 66407.1445163571 66406.10099072987 66396.35572803208 66387.93761405942 66387.5479649625 66372.6200558351 66361.43536264315 66360.2406151342 66358.13481128476 66350.45982473617 66349.6802717238 66349.48495499868 66317.18104798981 66316.99767392308 66316.94726493293 66316.85564549555 66314.33085466122 66312.72773533105 66300.06614476965 66295.09107017024 66294.4421000883 66291.4464296331 66274.47912448535 66274.0032011566 66271.86561151971 66269.8655832008 66264.76904996124 66263.77246015909 66260.50102375844 66258.83313878426 66254.50688353593 66196.68226622618 66189.51708539957 66188.04304862891 66187.44901666405 66142.14839162753 66136.44923415697 66134.52338268331 66134.33075359148 66133.75116192708 66113.84712136905 66104.93160246413 66103.64922652006 66102.43735800973 66100.03642725898 66087.0529711706 66086.03586990398 66085.35520876916 66085.34685470544 66083.80352503457 66083.13461844213 66078.91077668214 66078.65517716027 66078.43255881287 66071.53081073402 66071.15637591748 66061.65250441071 66061.24406522169 66060.88632441587 66060.80834511096 66059.60471317118 66054.08450471621 66053.8360854242 66050.7969013156 66047.98951924656 66047.88038673076 66047.83459353622 66047.6572104447 66047.54389039271 66047.44882258064 66047.0442023817 66045.56355329033 66045.48379517115 66045.06084163826 66043.7265067791 66043.29773720405 66042.56236704305 66011.35083534614 66007.77609490375 66006.82433252028 66006.76894164509 65993.74290645543 65987.15327663225 65983.46972614584 65981.92575941418 65981.06183469409 65978.33688096111 65978.25903455463 65977.92096935255 65977.76898802545 65974.68501555086 65973.74159294713 65973.13779435669 65973.09811699697 65970.36973875396 65965.39318318697 65964.83210597977 65949.31315811501 65948.17462649943 65947.69663066175 65944.98544921662 65944.5300514644 65942.17127988323 65940.85368970533 65938.38549892737 65938.1215899154 65935.68873624703 65934.61296851138 65933.18960214003 65929.21866467177 65925.28145097703 65925.10510092655 65923.42651769072 65922.76848187721 65920.355150313 1000 2000 3000 65917.77366988536 65917.42124579384 65917.35472768181 65915.191683115 1000 2000 3000 65913.74173835457 1000 2000 65913.67071983297 65913.54663343848 65913.38135904584 65912.91251332521 65912.86595519359 65912.38138264038 65911.87388184865 65911.0422651884 65910.91262789708 65909.44643254887 65907.18517001916 65906.6905428516 65906.63047319464 65903.66367389775 65903.04345749777 65900.11680201309 65898.03444366314 65897.89127352579 65897.72474233736 65897.42686146114 65889.75761242646 1000 2000 65889.4503775269 65888.60005291186 65887.96757268206 1000 2000 3000 65887.78502027241 65887.6735900183 1000 2000 65883.30746975438 65879.42451526482 65876.62593266835 65867.54309209646 65866.8705382418 65864.70234736537 65864.63872184069 65861.65396520337 65861.56001467745 65860.69255144284 65857.91063649424 65857.63904245653 65855.83938529169 65852.53674203974 65851.75722372605 1000 65849.42529692323 65849.39129165064 65849.21964358159 65849.13550889574 65849.01722979212 65848.309837367 65848.0190122691 65847.31340942821 65846.87780415732 65845.22126533584 1000 65844.68403011608 65844.50079455215 65843.99443125122 65833.12417188319 65823.5528378201 65820.2217303047 65820.11724628639 65815.84208487211 65815.59498372914 65812.9306441441 65812.68960067733 65809.75631444433 65809.49070219029 65809.40824727764 65808.68961578053 65807.856322961 65806.8239693821 1000 2000 65805.10277843967 65805.03041109092 1000 65804.22582580996 1000 65802.76896417789 1000 2000 3000 4000 65802.76660850768 65802.712087849 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000 51000 52000 53000 54000 55000 56000 57000 58000 59000 60000 61000 62000 63000 64000 65000 66000 67000 68000 69000 70000 71000 72000 73000 74000 75000 76000 77000 78000 79000 80000 81000 82000 83000 84000 85000 86000 87000 88000 89000 90000 91000 92000 93000 94000 95000 96000 97000 98000 99000 100000 15356.835419 seconds (2.08 M allocations: 2.805 TiB, 1.15% gc time, 0.00% compilation time)
(65802.712087849, [2.1141975308641965 0.0006598046978094491 0.11655773420479303 0.012777777777777782; 0.015043547110055414 0.000603697790014729 0.009920357691770257 0.0020190023752968953; … ; 0.252222222222222 0.0006175771971496377 0.0974509803921567 0.012699999999999986; 0.06587066082615065 0.0009636113373857049 0.022799624300758936 0.0048664688427299185], [2, 4, 2, 2, 4, 2, 2, 2, 2, 4  …  5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

65921.67410341187 [2.1141975308641965 0.01271420674405749 0.0006598046978094491 0.11655773420479303; 0.25152017689331124 0.012672953639761371 0.0006145046737807324 0.09725880401911995; … ; 0.7919389978213507 0.013071895424836607 0.0009780634343998867 0.14725105728565924; 0.0659648627007682 0.004859129532498635 0.0009626044289872564 0.022774688510419083], [3, 2, 3, 3, 2, 3, 3, 3, 3, 2  …  5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

### This is the output that the above line would give
87167.71403596654 87149.61885406218 87148.22380765609 87137.3598778919 87121.08450287407 87107.49513358274 87089.95521100592 87067.22995523066 87052.42110819285 87033.77503675935 87014.16041146395 86990.95315372797 86981.13212814093 86958.77089993196 86944.27284229794 86935.87397400678 86917.47202784251 86916.74157582349 86897.942167008 86882.51817746961 86873.79466863033 86873.2255956182 86854.54367711033 86843.87163316063 86827.37846181304 86807.8140260719 86802.78642604398 86786.98998096393 86775.48052296237 86756.62538460981 86737.6565426398 86723.53625220232 86719.0358606693 86711.3272902198 86693.42959895938 86676.7811185076 86654.84144016127 86636.91262302686 86624.56779982938 86603.95608740965 86581.21764476171 86562.2123948964 86542.25617678998 86523.72551719293 86507.42913447478 86488.53174148615 86471.28009163613 86470.28453385635 86453.39969779807 86449.97760778978 86446.62987768483 86427.57282611691 86405.44950754986 86383.13600645104 86360.05754275116 86344.10307452256 86320.55333416148 86319.10074547828 86300.78081766663 86278.03511877089 86255.91857782418 86232.92790463418 86231.41811084808 86223.49173332578 86204.26294299567 86185.96080498993 86164.47679715225 86144.34686904722 86120.5195254958 86100.11652467874 86077.95143727778 86062.62772052718 86058.82515073133 86058.11223829188 86036.0253509285 86012.50491928413 85988.82669877828 85973.12024838332 85957.39600828306 85936.45462461471 85930.01909373583 85911.35882489545 85888.19301704575 85868.69325092186 85847.9718765884 85823.89010478681 85799.13486552275 85775.45404618033 85774.23200159606 85763.123667893 85753.87042197675 85728.79015925525 85706.66020330759 85682.66871688623 85677.16397827596 85653.06465176311 85629.82027499902 85628.86781360637 85604.5548980143 85582.93637571349 85563.00914124158 85543.80168923474 85519.21735303767 85508.70690338955 85507.58680781185 85483.77008354448 85466.43973471073 85442.49995324542 85426.42067673136 85408.27310740697 85402.1175365539 85391.02973982232 85379.92963322099 85361.0062355132 85355.04860045738 85332.15835034962 85312.7380270489 85297.7488903285 85292.35422794214 85267.16796851566 85242.82859387898 85217.46209313608 85203.39225107034 85177.12545998767 85176.7870912172 85156.20926661488 85133.06242448997 85126.2109300678 85103.58580122956 85077.77158887523 85053.09758626482 85027.11014513765 85004.99831114085 84997.28306014171 84970.46192089458 84946.74033634509 84921.67592050991 84911.852391156 84891.38958791652 84871.79085001812 84852.88622584227 84829.36452162777 84811.96706673999 84787.38540602036 84761.52049472158 84758.22452163088 84753.03892735996 84727.28060026935 84700.87423693291 84695.79715730867 84669.8568581013 84647.04158779424 84629.04481181609 84606.260707402 84605.5083138072 84578.53123525668 84566.28418668175 84539.79639717996 84519.85097051282 84491.65119311804 84467.24300915224 84444.87653301487 84416.42145004222 84401.82134325331 84374.3272006225 84347.7067807016 84330.30999181622 84303.99933220081 84276.41943029783 84248.93174884429 84223.03062218022 84200.10852094131 84199.4707165633 84174.82580360076 84173.85531137718 84146.3348395354 84128.73011486737 84123.76539719992 84102.6144558432 84085.03092905869 84057.15893744535 84033.74954606511 84005.67353005311 83980.01263594512 83955.39915794235 83932.24195651895 83910.3276297308 83900.51645803779 83879.43364683898 83852.62958642124 83823.95690436049 83798.7366519996 83771.00789992248 83749.87753024005 83720.62541438876 83719.90185445028 83691.77354934927 83665.33009794973 83643.39905490288 83624.9293861102 83609.3241454376 83608.69198657945 83579.28106415551 83578.42500098677 83564.12024249756 83553.62098350374 83532.93766053191 83524.43371585393 83511.77884380914 83494.07971107976 83493.75252709181 83471.52447784165 83458.22906458183 83431.23897913707 83415.70030283011 83409.71827791125 83383.2375429431 83382.55292438657 83376.82434373863 83347.36072957783 83316.85796515188 83290.39132331235 83266.7396571635 83255.04953167272 83225.49660109801 83209.89735265508 83184.30343608938 83182.30840430922 83150.72422973378 83127.39879889667 83101.57748875639 83072.29854797687 83071.17031701037 83056.86109065848 83035.58235670268 83035.11461224114 83034.51398385795 83033.832588197 83005.7646873461 82997.86374633522 82969.74764808267 82941.48714144852 82917.20321084428 82896.76213213471 82896.15362504561 82867.76382733404 82867.20116048784 82841.19151394516 82840.94589880004 82808.2630157521 82807.301837701 82806.89315518684 82775.47233259236 82746.37601054572 82745.10333986602 82712.08814033931 82688.28319243946 82687.25200089884 82659.57616752625 82656.24336731424 82623.86622229694 82591.36284751199 82590.81602313202 82572.01127718395 82551.51271843073 82528.09196841644 82527.47207819043 82497.54685254139 82466.73144094515 82434.36369381136 82409.50421752616 82409.06585916114 82375.86767098158 82341.75170408621 82312.48383688636 82283.27144504001 82282.36548407833 82281.8804017306 82250.47520026824 82219.1510733262 82201.95671856105 82168.69553770073 82167.9143438016 82133.22208519849 82101.37800648573 82099.29750870114 82096.38327407966 82069.45642113202 82035.37665589554 82001.64657078864 81969.7201508475 81939.87697790703 81906.30504121268 81904.85469642388 81874.601670498 81842.2022399459 81808.61822740256 81801.32984946352 81772.0120540891 81764.12847013508 81746.75758613292 81745.46531936753 81712.62325564628 81710.50420122164 81675.68362524659 81643.76450744335 81643.16836093797 81641.4997229855 81606.00655266586 81605.3203411238 81580.71163877835 81580.07429503724 81580.05352380521 81578.85953245158 81546.6048847676 81511.51712143958 81479.10129012981 81476.60648663204 81470.12431944847 81469.51131748322 81467.61146855965 81437.97525018107 81406.31914520595 81369.98717895494 81368.78489131149 81346.84352819093 81346.47162020933 81328.40869048536 81292.64334144587 81263.96771944246 81228.48779342615 81217.85436095535 81181.7226812422 81146.00323659834 81145.02685765873 81116.97645690675 81093.03472376234 81056.96185854456 81020.84007140945 80989.06207995834 80953.60657409397 80952.05422999628 80950.3897577763 80949.56229674505 80949.45165746877 80939.89810809433 80935.46065331888 80921.97515179 80921.14291778074 80918.92024233291 80889.45818915898 80854.22292719544 80821.73244768009 80819.39812458506 80794.34539783251 80767.54879298926 80747.42251593166 80747.26225211669 80724.23608562564 80706.32965465085 80705.9341871948 80696.09940059166 80656.20762278003 80626.68610232887 80606.37436496346 80594.00826286754 80553.6946108771 80520.38877896205 80519.53002712085 80484.62581899302 80482.69474870063 80452.02930044908 80412.46543048644 80375.3891822471 80366.38055797822 80365.00120815258 80329.65282138411 80328.53275853884 80324.00829698572 80300.65513889903 80260.43694232195 80220.44366009349 80185.20244367947 80184.7545193642 80184.57117409572 80163.47853408167 80126.03047867675 80122.95068231008 80083.19565088637 80048.38806134253 80043.11854028798 80005.27628399542 79972.70249029304 79970.43994076803 79968.5030177153 79930.11929659468 79899.28872997171 79873.12011695432 79869.56713304365 79868.69797718283 79827.98666907047 79794.58321496187 79792.96035767556 79791.11341253236 79747.83003110663 79746.71108752489 79734.33636106378 79713.81690170303 79706.9250508427 79666.84659240961 79636.90598703823 79635.66218802157 79629.79945336762 79614.60138949259 79578.8656001348 79535.1459625492 79533.37580454253 79491.36086545375 79451.06658117793 79450.31276429082 79449.6591425425 79448.72210534106 79448.03844018147 79424.89716084914 79382.92742985295 79354.26235711276 79354.05392161744 79352.45015215286 79349.96712999007 79318.81794060492 79276.79456420512 79274.4649415061 79272.86687101127 79227.33916494783 79226.19527672025 79180.48974597106 79145.33799466377 79103.9038405545 79085.93642391029 79040.78422192624 79001.48471874117 78998.6876173976 78997.83875833756 78996.46601784334 78996.3006623107 78995.91515296094 78954.89154950873 78954.1227626199 78914.39121190405 78905.30790033717 78905.23578573586 78877.5376803856 78835.20944886185 78797.95959627576 78754.6955611955 78711.89119950822 78707.17571589146 78660.30424985505 78658.25108053636 78656.1074809516 78656.0725353308 78628.8420644258 78627.71378778489 78627.27636424151 78583.67731825782 78550.0005792197 78504.67405089871 78489.79815222838 78489.5963830036 78448.58086460718 78402.59495008316 78389.7975118163 78347.14820452796 78322.50605690268 78322.01310231867 78273.30581384379 78273.1765221043 78269.14264634863 78222.36626706469 78219.53973172937 78176.73417030658 78176.47326929304 78169.02965730829 78152.68756243541 78152.4996763453 78151.74618718498 78145.11465042902 78110.33898810555 78072.31126140502 78035.83758056765 78034.02205302134 78033.74625544777 78031.79033916564 77987.56611871826 77945.37760793102 77943.31411505608 77891.9676374851 77886.82857598913 77837.68716788029 77785.91676166252 77745.57498301449 77739.2976925099 77689.563686873 77688.78113527375 77687.42782265817 77644.47205559682 77641.89167254575 77641.33447808924 77641.22160041668 77598.69004508332 77597.08379754174 77597.08356554018 77577.07861781714 77532.90007682425 77532.8443103482 77529.96973737994 77525.55981076401 77479.53939128465 77446.04195171037 77434.40198950471 77388.71789787618 77387.8902014261 77387.03260387201 77386.08858331632 77385.44231604233 77340.74450695362 77298.21188876092 77243.53668329945 77228.78710220329 77181.2225357782 77180.25569562917 77178.5330615377 77178.12410783661 77175.73971012117 77123.64106624249 77112.47162969763 77102.64379981757 77100.0893755922 77096.0737433437 77040.07092023564 77039.0622657533 77037.9117094408 77007.72734481299 76956.32232171745 76949.49544609184 76897.8248601771 76882.56036953392 76882.09991625296 76880.230366896 76876.49916571895 76876.05694207504 76875.58310820113 76857.09180989079 76854.94844233656 76840.84268374812 76839.35277402533 76835.37103100953 76818.51707834104 76815.06741337001 76813.870744453 76792.96843455316 76791.81871062967 76790.06759449525 76754.15512380034 76753.125570011 76744.55001879344 76743.92916541653 76714.89854621216 76671.11631595244 76637.74971618367 76602.65440446371 76601.50034345294 76585.63976087018 76582.05735726013 76538.51249665343 76534.9466508297 76534.11754583461 76533.36148936802 76530.43222475413 76525.52602041309 76511.93306572587 76511.14996910655 76487.03108703713 76438.09755372451 76437.03122255302 76427.11621704312 76390.54060615039 76389.19717055585 76384.1674447534 76377.62430718089 76377.15694596067 76357.90994237163 76357.7622353188 76352.64310431959 76291.64783066108 76290.00991411797 76270.1489949741 76269.6716641967 76269.47571305132 76264.20291456535 76207.94424673473 76201.75972438105 76145.8280615389 76141.47736129716 76126.20489153023 76125.80479636467 76122.30257358478 76121.07732667336 76118.60513137247 76117.35105501622 76116.78885078723 76112.18224681747 76110.93978608842 76109.85507261484 76053.90448930982 75991.3554781675 75942.38330005144 75927.71944017966 75927.68889022172 75923.33371219586 75914.77453009524 75912.25917911823 75900.14141968887 75842.21329909169 75779.97367581615 75723.35113594522 75689.50735030485 75689.00240594805 75687.69692035696 75686.34893591588 75676.12469879992 75669.2783088293 75667.72894359514 75643.44291563616 75638.78335663918 75579.40251469864 75576.83691485447 75573.48849042178 75555.40290986664 75550.00782710727 75527.80180780341 75526.98272045417 75467.25202497125 75463.89322566966 75409.21662876953 75408.88387351458 75342.26336780516 75338.21209649881 75335.83617071403 75334.23620446886 75267.24972514373 75208.02448967625 75205.31223363294 75200.57202071778 75199.61986774938 75198.43493985878 75196.62852880446 75193.04301023005 75190.18992399631 75159.99669729442 75138.94895708929 75109.34967507033 75103.1331013997 75098.38234976167 75048.81598353412 75025.1722956521 75024.65972203288 74957.18641705834 74920.91553357126 74919.40873872634 74875.37963396599 74867.3799650937 74862.06646641645 74859.04428620538 74852.37975317649 74849.04966765924 74785.37789264161 74775.46284666481 74772.23839619584 74736.94436917614 74735.19997123502 74675.18825753064 74673.46415737622 74669.68448651729 74604.5283931722 74541.16036317323 74540.8364389864 74537.9161986723 74534.962304261 74522.3536318142 74522.19184908642 74500.25495791428 74491.01028006557 74486.09873644562 74485.03869457514 74471.76611368063 74469.89167284082 74412.84574562867 74408.59727279154 74390.07770513878 74387.74121699297 74322.67415781757 74319.4933762313 74296.06486228958 74294.9431063713 74246.48297452171 74225.65374187811 74223.20305457395 74222.8973315545 74204.93507516304 74136.19986029493 74133.6809387133 74060.02842902242 74058.51739445537 74055.6575193387 74055.27334133384 74052.76079747747 74052.1296744281 74048.7446846814 74042.6668389537 74042.66320613821 74015.06823810842 74009.90845806163 74009.08704931801 74008.69496031338 73994.36565671444 73993.36238582923 73991.39102049262 73988.81562893449 73969.75418879949 73941.84693325704 73935.45305922799 73913.46519831615 73909.72782300302 73908.50447927028 73903.79277250521 73838.29068550168 73761.69214814853 73753.45835551356 73753.16636618008 73751.06867160035 73748.46187480041 73747.74710544119 73745.08924384718 73744.46519655742 73739.50057674627 73729.0928664609 73726.99808484738 73721.91021367963 73721.2691313403 73715.84072763838 73713.910549784 73700.76904745992 73683.89204355559 73682.82237807158 73610.7762424235 73599.08399479781 73534.57200235219 73533.71747859613 73533.42697469855 73528.38832692575 73521.10628919031 73519.86089129385 73513.2597783443 73507.76151263212 73502.27425631104 73499.48205073709 73464.40589715687 73442.00483375299 73437.94063796292 73371.03145899606 73351.43865771734 73340.14022193594 73309.41246089594 73308.69401322103 73306.03867105056 73301.05875590828 73300.95644231295 73275.56271219709 73271.70997718888 73258.10548496117 73257.473399279 73191.31403923777 73187.56313519708 73165.63957734598 73165.57576253758 73124.48793985424 73112.89523418926 73111.12157800136 73109.65664385707 73102.96716564205 73094.65898623114 73094.23165742397 73092.34329005906 73090.05690366865 73087.88576567463 73072.38363530682 72998.62072958228 72953.12250958676 72952.72807469577 72946.2396315106 72858.47468027144 72855.8525268395 72853.26800220706 72844.80160657382 72843.89501255969 72841.40662923777 72821.34076812549 72819.93267211205 72814.4250272883 72814.39900417061 72811.58375961107 72796.47006151003 72795.5184078073 72787.8295204193 72784.94013704949 72784.69281541328 72771.44529611693 72770.37882566922 72767.54429085697 72758.97261599245 72758.2395168408 72754.6777196157 72700.53934512823 72672.09466452578 72593.12834125746 72591.64923009383 72512.02621993421 72510.70245087246 72509.79177905586 72496.15286906049 72494.39578530972 72486.82435684024 72480.74732504226 72475.07109529823 72465.40473602201 72464.42900753528 72463.46911693575 72460.96834812404 72459.67031488223 72451.86406055593 72440.35516465708 72437.56165562387 72434.57864561943 72431.98879690205 72431.92761406642 72414.86953865174 72407.60761064268 72396.26808156974 72318.4277998456 72279.41780297345 72271.35954219018 72260.05649552072 72258.4843710004 72257.75032759582 72253.85985654853 72207.25108461475 72201.89947415669 72120.20908925538 72085.35094359965 72081.21614193352 72063.05045538799 72059.42353824488 72016.28244169433 72005.15095629361 71997.98138367511 71967.2207372804 71961.54787713978 71949.22347902035 71944.93749164585 71942.36851390793 71940.11982817123 71839.67351734525 71835.8845217403 71828.01208268202 71824.33393816504 71788.95942522898 71787.25811284284 71784.87206845825 71780.05459070971 71763.07344798058 71685.62569055363 71666.18612173929 71648.20517504617 71641.090899901 71639.3890178575 71635.50989471047 71631.59722311689 71631.4651312998 71531.92178154316 71530.15252939322 71530.02110969348 71527.31663335636 71476.20653087263 71411.24804022037 71378.55546124076 71375.47593263163 71371.4076242733 71368.46569647845 71360.69187950762 71352.17112201583 71351.33606860624 71335.87047346715 71327.8346407129 71319.50615511114 71296.95404459999 71193.15539965608 71161.46778382437 71158.4104877191 71156.3462016166 71148.347953868 71134.28202283656 71109.4943039551 71106.4425997149 71075.30274133515 71013.17794330865 71010.91355309082 70899.87030999162 70896.00452049365 70853.83250768937 70843.06256839156 70839.50790742243 70836.23313982096 70832.92078993062 70828.54397293323 70812.07567261271 70803.3044193912 70799.896139303 70791.98399810668 70788.53674994012 70788.25567420284 70783.40961789162 70780.11792320416 70772.69423379445 70765.93670433541 70762.7469571954 70740.5312451535 70734.60714303242 70720.3854747955 70712.26812858213 70711.31885645501 70707.22263423688 70677.3344772791 70668.07639809902 70639.86741549519 70607.57941974862 70603.74374064356 70568.83424600241 70538.77882211423 70535.1034509543 70531.3783742556 70529.67850826622 70506.89328287022 70502.37109871153 70499.10192567033 70497.62774738473 70486.10274052912 70469.09678903011 70465.79759063467 70463.29123509185 70458.41061510413 70458.30071050025 70447.65840496434 70425.7202496264 70349.47461620488 70331.63516780105 70219.38564784364 70173.250817538 70169.45162035424 70143.45602986524 70073.32437784011 70068.81111717132 70067.24501194751 70050.40658903039 69980.75396884145 69976.15370471432 69974.60064230737 69972.21984365121 69967.56056151427 69939.06133747559 69936.67588446567 69890.70747168051 69888.40524213466 69861.44273497096 69843.49245130512 69837.85629265742 69833.45917805817 69832.07045525618 69828.06121448941 69809.23514111832 69806.39400518245 69804.97035648373 69800.33719531518 69796.52807764735 69794.02264663207 69790.46596975636 69783.54561607634 69779.55341935823 69774.35081717368 69774.11336967013 69748.28305957702 69710.95170979359 69641.83478833259 69603.5298063238 69598.72080484577 69587.12247110126 69582.52834859569 69578.8348768177 69545.27250685966 69519.47259442409 69512.72499313817 69508.75504349815 69507.12224353323 69506.98188166192 69503.90398009474 69498.58244497767 69493.68728724169 69492.92256759625 69489.18218820156 69488.2098062019 69482.71812374092 69461.43795861996 69460.59461943235 69440.49304237831 69439.38256355358 69414.27285745108 69413.63255691611 69411.42027577639 69406.3789364394 69402.91096827434 69376.16497153004 69374.92052586308 69371.0566022293 69370.66954472507 69364.9769783633 69353.87370858599 69328.22376963189 69327.07731112515 69326.12173019849 69325.92164949066 69291.44783645435 69271.35632467683 69266.23411206914 69264.22896297365 69263.81852618272 69177.10893239737 69165.86175157146 69151.32417187568 69145.82755077674 69139.93392804312 69139.88069431565 69133.89643242172 69098.09665434061 69092.13880058807 69084.59012195567 69080.6283055683 69074.55965649607 69067.57693653519 69054.92780834714 69022.55253839605 69019.29382727308 69015.62480985487 69014.3148409516 68968.1969411958 68960.46904280993 68956.65165896088 68908.82751574402 68908.01098111196 68902.80066931153 68896.38190326568 68895.35875203782 68891.3825488788 68888.75202029281 68862.87681482204 68848.78613431158 68811.62685266782 68805.30221421822 68804.78082192421 68800.35784312965 68780.14713367958 68717.58952458874 68694.2804116369 68687.42849802975 68683.08241357651 68677.54891364125 68660.31776039267 68604.57238818258 68598.38026862749 68598.20303328645 68594.65487077157 68572.99630330858 68571.29915001354 68518.91270508293 68471.31692149548 68460.30031988904 68457.0754469166 68456.36146222959 68450.05278094021 68438.4846007774 68437.72527322867 68432.35454385151 68425.9191163563 68424.86384862846 68290.23420045868 68289.64347410057 68255.6257505117 68238.68421993474 68237.92926773298 68235.43039679166 68174.52287431629 68164.18466422161 68160.43015012983 68083.53629981678 68079.22558938916 68073.02802973024 68027.32665615046 67992.6532218189 67982.23331008821 67975.51775388436 67950.82702996908 67926.39269491694 67919.61588788702 67919.26753857678 67917.77189020041 67908.98889606341 67825.12939690061 67820.80707977212 67811.48277796447 67804.96701692272 67800.94512679089 67800.11329780893 67792.44065103742 67785.67119964794 67784.1068340844 67783.52239641907 67780.4691319763 67778.40532222514 67770.47492583324 67766.65917426728 67765.83349118527 67755.0808179818 67731.95273686269 67731.30723284364 67730.26163160316 67726.5784390177 67726.1363005064 67724.00944075998 67719.15102437862 67715.38023025503 67680.74963965091 67661.17894948272 67657.44371158897 67638.57046179465 67638.17358441533 67633.52619681714 67632.0218946069 67620.5549762072 67596.0690437576 67581.7227957099 67558.34029157642 67552.51449098093 67548.24013120348 67547.99568746172 67547.31655140681 67537.91663808761 67537.68088557968 67536.638531262 67534.90658520968 67530.75478202397 67527.61454678267 67526.6048664426 67522.91123567324 67512.34210076812 67509.81625600871 67508.0480874164 67500.26532850147 67496.19142487767 67494.65151223193 67486.8939525352 67466.1014314649 67463.77457460697 67456.44971302309 67448.25187412316 67440.84086056193 67435.0930074908 67431.4034178892 67429.20774997748 67424.13288540616 67421.92849977061 67421.55589826498 67416.3285630802 67415.8812169038 67411.70410222 67404.19959006413 67391.48146136347 67388.4734613088 67376.6916591509 67375.39463951951 67327.050439479 67313.01072102076 67310.87721765417 67310.82483213526 67310.69147357014 67307.23229622944 67306.85074216523 67306.5617492245 67255.4156420599 67249.65631628642 67246.0559890093 67243.72164951434 67243.14052358594 67201.58399271464 67200.72368828661 67194.17917215174 67185.93396221615 67183.2131976372 67154.07902751456 67146.65102403147 67139.16631791653 67138.78945292011 67107.08474284351 67103.24178776132 67102.52767772129 67100.3812238902 67096.73636607843 67095.79387365219 67087.26371374469 67081.155248661 67077.15774598096 67031.79248444168 67016.82443845547 67013.81398378377 67009.20890971832 67004.82143926396 66997.53664262267 66996.41269181564 66992.3431404683 66990.90341133234 66941.83700292249 66938.03667762999 66931.75602757726 66931.74386998218 66926.69147564491 66921.68541134374 66913.12817672934 66907.09115336805 66905.61972446072 66905.43726948349 66905.21540071264 66881.13104702294 66879.04809475406 66870.87545797578 66854.97558241677 66849.16812453017 66848.69035199926 66842.95077380684 66842.54037061051 66840.51399211341 66840.42479442523 66832.27412747202 66827.86137244888 66825.12499373077 66816.92393949628 66809.33482160495 66804.72777814027 66800.70233395329 66800.03461732177 66799.73159453682 66794.35506050466 66752.99426560116 66752.3412882237 66744.75068140657 66741.56407669905 66741.47734069856 66712.09543789522 66697.94778665525 66696.565616747 66696.49438016431 66689.91490196668 66684.1758472892 66684.01772441779 66675.7784633123 66661.93878345129 66660.885640425 66655.87452423248 66653.0794676184 66650.63428294618 66650.40238331766 66649.51366600457 66647.80382892142 66647.6596788887 66638.82294677602 66638.45283849142 66638.099302394 66629.0940151145 66628.70227417498 66626.93260313319 66619.82571723375 66619.30992517632 66615.63062178982 66610.40530991426 66571.1375270656 66565.61325431366 66565.58130610436 66563.52091834613 66563.12821159726 66561.94242217869 66561.73668985748 66560.98532739887 66560.96767640345 66560.5601299048 66555.1863847816 66553.4523748752 66549.48355191578 66540.65445783988 66538.96008169625 66537.5413066855 66537.25547949782 66528.2620826575 66521.79494158152 66518.07165268098 66503.64333850604 66502.77985403335 66500.55845244706 66482.95311453166 66482.82975249676 66476.18465918035 66474.88755441332 66472.62534635441 66471.50843545314 66466.05922103555 66465.6768695188 66461.01601204916 66455.18916366335 66452.9304424318 66445.34909676903 66438.39044889805 66429.41643594715 66423.86370598934 66422.06257689037 66421.31437537997 66420.7101397019 66411.37407166514 66409.74062124953 66409.10232364596 66407.1445163571 66406.10099072987 66396.35572803208 66387.93761405942 66387.5479649625 66372.6200558351 66361.43536264315 66360.2406151342 66358.13481128476 66350.45982473617 66349.6802717238 66349.48495499868 66317.18104798981 66316.99767392308 66316.94726493293 66316.85564549555 66314.33085466122 66312.72773533105 66300.06614476965 66295.09107017024 66294.4421000883 66291.4464296331 66274.47912448535 66274.0032011566 66271.86561151971 66269.8655832008 66264.76904996124 66263.77246015909 66260.50102375844 66258.83313878426 66254.50688353593 66196.68226622618 66189.51708539957 66188.04304862891 66187.44901666405 66142.14839162753 66136.44923415697 66134.52338268331 66134.33075359148 66133.75116192708 66113.84712136905 66104.93160246413 66103.64922652006 66102.43735800973 66100.03642725898 66087.0529711706 66086.03586990398 66085.35520876916 66085.34685470544 66083.80352503457 66083.13461844213 66078.91077668214 66078.65517716027 66078.43255881287 66071.53081073402 66071.15637591748 66061.65250441071 66061.24406522169 66060.88632441587 66060.80834511096 66059.60471317118 66054.08450471621 66053.8360854242 66050.7969013156 66047.98951924656 66047.88038673076 66047.83459353622 66047.6572104447 66047.54389039271 66047.44882258064 66047.0442023817 66045.56355329033 66045.48379517115 66045.06084163826 66043.7265067791 66043.29773720405 66042.56236704305 66011.35083534614 66007.77609490375 66006.82433252028 66006.76894164509 65993.74290645543 65987.15327663225 65983.46972614584 65981.92575941418 65981.06183469409 65978.33688096111 65978.25903455463 65977.92096935255 65977.76898802545 65974.68501555086 65973.74159294713 65973.13779435669 65973.09811699697 65970.36973875396 65965.39318318697 65964.83210597977 65949.31315811501 65948.17462649943 65947.69663066175 65944.98544921662 65944.5300514644 65942.17127988323 65940.85368970533 65938.38549892737 65938.1215899154 65935.68873624703 65934.61296851138 65933.18960214003 65929.21866467177 65925.28145097703 65925.10510092655 65923.42651769072 65922.76848187721 65920.355150313 1000 2000 3000 65917.77366988536 65917.42124579384 65917.35472768181 65915.191683115 1000 2000 3000 65913.74173835457 1000 2000 65913.67071983297 65913.54663343848 65913.38135904584 65912.91251332521 65912.86595519359 65912.38138264038 65911.87388184865 65911.0422651884 65910.91262789708 65909.44643254887 65907.18517001916 65906.6905428516 65906.63047319464 65903.66367389775 65903.04345749777 65900.11680201309 65898.03444366314 65897.89127352579 65897.72474233736 65897.42686146114 65889.75761242646 1000 2000 65889.4503775269 65888.60005291186 65887.96757268206 1000 2000 3000 65887.78502027241 65887.6735900183 1000 2000 65883.30746975438 65879.42451526482 65876.62593266835 65867.54309209646 65866.8705382418 65864.70234736537 65864.63872184069 65861.65396520337 65861.56001467745 65860.69255144284 65857.91063649424 65857.63904245653 65855.83938529169 65852.53674203974 65851.75722372605 1000 65849.42529692323 65849.39129165064 65849.21964358159 65849.13550889574 65849.01722979212 65848.309837367 65848.0190122691 65847.31340942821 65846.87780415732 65845.22126533584 1000 65844.68403011608 65844.50079455215 65843.99443125122 65833.12417188319 65823.5528378201 65820.2217303047 65820.11724628639 65815.84208487211 65815.59498372914 65812.9306441441 65812.68960067733 65809.75631444433 65809.49070219029 65809.40824727764 65808.68961578053 65807.856322961 65806.8239693821 1000 2000 65805.10277843967 65805.03041109092 1000 65804.22582580996 1000 65802.76896417789 1000 2000 3000 4000 65802.76660850768 65802.712087849 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000 51000 52000 53000 54000 55000 56000 57000 58000 59000 60000 61000 62000 63000 64000 65000 66000 67000 68000 69000 70000 71000 72000 73000 74000 75000 76000 77000 78000 79000 80000 81000 82000 83000 84000 85000 86000 87000 88000 89000 90000 91000 92000 93000 94000 95000 96000 97000 98000 99000 100000 15356.835419 seconds (2.08 M allocations: 2.805 TiB, 1.15% gc time, 0.00% compilation time)
(65802.712087849, [2.1141975308641965 0.0006598046978094491 0.11655773420479303 0.012777777777777782; 0.015043547110055414 0.000603697790014729 0.009920357691770257 0.0020190023752968953; … ; 0.252222222222222 0.0006175771971496377 0.0974509803921567 0.012699999999999986; 0.06587066082615065 0.0009636113373857049 0.022799624300758936 0.0048664688427299185], [2, 4, 2, 2, 4, 2, 2, 2, 2, 4  …  5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [19]:
function bucket_extract(assign, A::Matrix{Int64}, num, numsink)
    T = zeros(Int64, num + numsink, num)
    for i in 1:size(A)[1], j in 1:size(A)[2]
         @inbounds T[(num + 1) * (assign[j] - 1) + assign[i]] += A[i, j]
    end
    return T
end

bucket_extract (generic function with 1 method)

The estimated means are:

In [20]:
est_mat = bucket_extract(est_alloc, out, NUMBER_OF_TYPES, NUMBER_OF_SINKS)

5×4 Matrix{Int64}:
  441   821   861   41
  108   263   615   15
   41    94   666    3
   96   178    93   67
 1208  1751  1949  438

The matrix is then ordered such that, for any two elements in symmetric positions across the diagonal, the element below the diagonal is greater than the element above the diagonal. For example, the 204 in cell (1, 2) is greater than the 50 in the symmetric cell (2, 1) below, as are all the other symmetric pairs.

In [21]:
sum(est_mat)

9749

In [22]:
M = est_mat

5×4 Matrix{Int64}:
  441   821   861   41
  108   263   615   15
   41    94   666    3
   96   178    93   67
 1208  1751  1949  438

The next bit re-orders the tiers so that tier 1 is in the first, tier 2 the second, etc.  The ordering of the tiers is by total placements.

In [23]:
# the new placements matrix
placement_rates = zeros(Int64, (TOTAL_DISTRIBUTIONS, NUMBER_OF_TYPES))
#row sums in the estimated matrix
ovector = sum(M, dims=1)
# row sums reordered highest to lowest
svector = sortslices(ovector,dims=2, rev=true) 
#println(svector)
#println(length(ovector))
# a mapping from current row index to the index it should have in the new matrix
o = Dict{}()
for i in 1:length(ovector)
    for k in 1:length(svector)
        if ovector[1,i] == svector[1,k]
            o[i] = k
            break
        end
    end
end 
#println(o)
P = zeros(Int64, (TOTAL_DISTRIBUTIONS, NUMBER_OF_TYPES))
#shuffle the cells for the tier to tier placements
for i in 1:NUMBER_OF_TYPES
    for j in 1:NUMBER_OF_TYPES
        placement_rates[o[i],o[j]] = M[i,j]
    end
end
#shuffle the cells for tier to sink placements (separate since sink row indices don't change)
for i in NUMBER_OF_TYPES+1:NUMBER_OF_TYPES+NUMBER_OF_SINKS
    for j in 1:NUMBER_OF_TYPES
        placement_rates[i,o[j]] = M[i,j]
    end
end
placement_rates


            
    

5×4 Matrix{Int64}:
  666    94    41    3
  615   263   108   15
  861   821   441   41
   93   178    96   67
 1949  1751  1208  438

In [43]:
println(sum(placement_rates))

9749


In [44]:
o # this is the type to tier mapping, first argument is type, second is the tier

Dict{Any, Any} with 4 entries:
  4 => 4
  2 => 2
  3 => 1
  1 => 3

To verify that this is ordered properly, we can check symmetric indices:

In [28]:
for i in 1:NUMBER_OF_TYPES, j in 1:NUMBER_OF_TYPES
    if i > j # not a diagonal and only check once
        if placement_rates[i, j] <= placement_rates[j, i]
            println("FAULT: hiring ", i, " with graduating ", j, ": downward rate: ", placement_rates[i, j], ", upward rate: ", placement_rates[j, i])
        end
    end
end
println("Check Complete")

Check Complete


If everything worked fine, there should be no faults in this cell. If there are, change the order in the cell with the explicit order.

James' code only matched an institution name with a tier.  To save the matching to the database (and to use it for other purposes) we need to match tiers with oids.  Two dictionaries were created above
`oid_mapping` has keys that correspond with oids and values that coincide with institution ids,
`institution_mapping` has keys which are institution ids, while values are the corresponding institution name
The next function reverses this, takes an institution name and returns the institution id, then the set of oids that are associated with that institution.
`false` is returned if either is not found.

In [29]:
function name_to_oid(institution_name::String, institutions::Dict, organizations::Dict)
    oids = String[]
    institution_id = String
    for k in keys(institutions)
        if institutions[k] == institution_name
            institution_id = k
            break
            #push!(oids,k)
        end
    end
    for k in keys(organizations)
        if organizations[k] == institution_id
            push!(oids, k)
        end
    end
return institution_id, oids
end


name_to_oid (generic function with 1 method)

The lengths of mappings.  The total number of institutions should exceed the total number in the institution mapping by the number of academic institutions who hired at a level other than assistant professor (postdocs,lecturer, etc).  The saved mapping the the database should then contain the same number as the institution_mapping.

In [34]:
println(length(institution_mapping))
print(length(institutions))

1451
2193

For mysql

In [35]:
#saving type allocation
function save_type(tier, oids, stm)
    for oid in oids
        DB.execute(stm, [tier, oid])
    end
end


save_type (generic function with 1 method)

In [41]:
o

Dict{Any, Any} with 4 entries:
  4 => 4
  2 => 2
  3 => 1
  1 => 3

In [37]:
#to record the type allocation
n = 0
B = Set{String}()

if SAVE_TO_DATABASE
    d = DB.connect(MySQL.Connection,cfg["host"], cfg["user"], cfg["password"], db =cfg["database"], port = parse(Int64,cfg["port"]))
    query = "drop table if exists type_distribution"
    DB.execute(d, query)
    query = "create table type_distribution (id int auto_increment primary key, type int, oid int,created timestamp default CURRENT_TIMESTAMP )"
    DB.execute(d, query)
    query = "insert into type_distribution set type=?,oid=?"
    stm = DB.prepare(d, query)
end
for j in 1:NUMBER_OF_TYPES
    println("Type ", j)
    println()
    i = 1
    for entry in est_alloc
        if entry == j
            push!(B,institutions[i])
            iid, oids = name_to_oid(institutions[i],institution_mapping, oid_mapping)
            println(institutions[i]," ",oids)
            if(length(oids)) == 0
                println("*****Error in data****")
                n += 1
            end
            if SAVE_TO_DATABASE
                save_type(j, oids, stm)
            end
        end
        i += 1
    end
    println(n," errors counted")
    println()
end
if SAVE_TO_DATABASE
    DB.close(d)
end


Type 1

University of Oslo ["3810", "675"]
Rice University ["2099", "110"]
University of Massachusetts, Amherst ["1635", "2250", "1642", "330"]
Eastern Kentucky University ["65"]
Peking University ["1830", "767", "3008", "1273", "2904"]
University of Lausanne (Université de Lausanne) ["376", "3355"]
Georgetown University ["6269", "1419", "99", "4600", "1184"]
Universidad de los Andes ["1033", "533"]
Vrije Universiteit Amsterdam ["898", "72", "250", "1369", "2252"]
Hong Kong Baptist University ["1080"]
Simon Fraser University ["332", "1310"]
Colorado State University ["4997", "1788", "3130"]
University of Leicester ["477"]
London Business School ["1754", "660", "3203", "3276"]
University of Amsterdam (Universiteit van Amsterdam) ["5211", "1343", "2389"]
Wuhan University ["1183", "4820"]
George Mason University ["2756", "1770", "1637"]
Drexel University ["2379", "634"]
University of Miami ["2546", "3527", "750"]
Concordia University ["160", "7009"]
University of Kentucky ["1535", "3924",

In [38]:
n = 0
B = Set{String}()
if SAVE_TO_DATABASE
    d = DB.connect(MySQL.Connection,cfg["host"], cfg["user"], cfg["password"], db =cfg["database"], port = parse(Int64,cfg["port"]))
    query = "insert into type_distribution set type=?,oid=?"
    stm = DB.prepare(d, query)
end
for j in NUMBER_OF_TYPES+1:NUMBER_OF_SINKS+NUMBER_OF_TYPES
    println("SINK ", j - NUMBER_OF_TYPES)
    println()
    i = 1
    for entry in est_alloc
        if entry == j 
            if occursin("(private sector)", institutions[i])
                ch = 17
            elseif (!occursin("(private sector)", institutions[i]) && !occursin("(public sector)", institutions[i]) 
                    && !occursin("(academic sink)", institutions[i]))
                ch = 0
            else ch = 16
            end
            if !(institutions[i][1:prevind(institutions[i], end,ch)] in B)
                iid, oids = name_to_oid(institutions[i][1:prevind(institutions[i],end,ch)],institution_mapping, oid_mapping)
                println(institutions[i]," ", oids)
                if length(oids) == 0
                    println("****** error in data*****")
                    n +=1
                end
                if SAVE_TO_DATABASE
                    save_type(j, oids, stm)
                end
                push!(B, institutions[i][1:prevind(institutions[i], end,ch)])
            end
        end
        i += 1
    end
    println()
    println(n," errors counted")
end
DB.close(d)

SINK 1

CNRS (Centre national de la recherche scientifique) (academic sink) ["270"]
De Nederlandsche Bank (academic sink) ["3357"]
Institute of Economics (Ekonomski institut), Zagreb (academic sink) ["3474"]
University of Bremen (Universität Bremen) (academic sink) ["7847"]
Texas A&M University, College Station (academic sink) ["1990", "596", "1823", "3003"]
Universitas Indonesia (academic sink) ["1544"]
University of Oxford (academic sink) ["1050", "1398", "1233", "612", "693", "1832", "1504", "5931", "217"]
Universitat Pompeu Fabra (academic sink) ["453", "998"]
City University of London (academic sink) ["883", "1094"]
Coventry University (academic sink) ["2996"]
Centre d'Etudes Prospectives et d'Informations Internationales (CEPII) (academic sink) ["1696"]
University of Sussex (academic sink) ["430", "2549", "831"]
Johns Hopkins University (academic sink) ["1043", "49", "1345", "2864", "2340", "2624", "1328", "537"]
State University of New York at Buffalo (academic sink) ["1756", "4

In [39]:
println("acd_sink ",length(acd_sink))
println("teaching ",length(tch_sink))
println("academic ",length(academic))
println(" gov ", length(gov_sink))
println("private ", length(pri_sink))
println("institutions ", length(institutions))

acd_sink 492
teaching 983
academic 468
 gov 114
private 136
institutions 2193


## References

Karrer, B., and M. E. J. Newman (2011): "Stochastic Blockmodels and community structure in networks," Physical Review, 83(1).

Peixoto, T. (2014): "Efficient Monte Carlo and greedy heuristic for the inference of stochastic block models," Physical Review, 89(1).